## Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install pydub timm resampy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_

In [ ]:
from pydub import AudioSegment

In [ ]:
import os
import torch
from tqdm import tqdm
import json
import time
import requests
import torchaudio
import numpy as np
import pickle
import cv2
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import math
import torch.nn as nn
import torch.nn.functional as F
import random
from torch import optim
from sklearn.metrics import f1_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
root = '/content/drive/MyDrive/MSc/Thesis'

In [ ]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True

In [ ]:
setup_seed(20)

# Dataset

In [ ]:
#test_list = []
a = []
for d in ['VA_Estimation_Challenge','EXPR_Recognition_Challenge','AU_Detection_Challenge']:
    data_dir=os.path.join(root,'data/Annotations',d)
    for k in ['Train_Set','Validation_Set']:
        data_label=os.path.join(data_dir,k)
        with open(os.path.join(data_dir,f'{k}.txt'), 'w') as f:
            for filename in tqdm(os.listdir(data_label)):
                fn, ext = os.path.splitext(os.path.basename(filename))
                if ext.lower()=='.txt':
                    f.write(fn+'\n')

100%|██████████| 105/105 [00:00<00:00, 110737.22it/s]


In [ ]:
test_list = []

for d in ['VA_Estimation_Challenge','EXPR_Recognition_Challenge','AU_Detection_Challenge']:
    print(d)
    with open(os.path.join(root,f'data/Annotations/{d}/Train_Set.txt'), 'r') as f:
        files = f.read().splitlines()
    random.shuffle(files)
    ratio = int(len(files)/5) # 20%
    val_set = files[:ratio]
    test_list.extend(val_set)
    train_set = files[ratio:]
    print('Train_set:')
    with open(os.path.join(root,f'data/Annotations/{d}/Train.txt'), 'w') as f:
        for ftrain in tqdm(train_set):
            f.write(ftrain+'\n')
    print('Val_set:')
    with open(os.path.join(root,f'data/Annotations/{d}/Val.txt'), 'w') as f:
        for fval in tqdm(val_set):
            f.write(fval+'\n')

    with open(os.path.join(root,f'data/Annotations/{d}/Validation_Set.txt'), 'r') as f:
        test_set = f.read().splitlines()
    test_list.extend(test_set)
    print('Test_set:')
    with open(os.path.join(root,f'data/Annotations/{d}/Test.txt'), 'w') as f:
        for ftest in tqdm(test_set):
            f.write(ftest+'\n')
with open(os.path.join(root,'data/test_list.txt'), 'w') as f:
    for fn in test_list:
        f.write(fn+'\n')

VA_Estimation_Challenge
Train_set:


100%|██████████| 285/285 [00:00<00:00, 300799.36it/s]


Val_set:


100%|██████████| 71/71 [00:00<00:00, 387250.43it/s]


Test_set:


...

100%|██████████| 236/236 [00:00<00:00, 772118.37it/s]


Val_set:


100%|██████████| 59/59 [00:00<00:00, 391556.86it/s]


Test_set:


100%|██████████| 105/105 [00:00<00:00, 522422.21it/s]


In [ ]:
def get_names(vid, id):
    name = ""
    if id>=0 and id<10:
        name = f"{vid}/0000" + str(id) + ".jpg"
    elif id>=10 and id<100:
        name = f"{vid}/000" + str(id) + ".jpg"
    elif id>=100 and id<1000:
        name = f"{vid}/00" + str(id) + ".jpg"
    elif id>=1000 and id<10000:
        name = f"{vid}/0" + str(id) + ".jpg"
    else:
        name = f"{vid}/" + str(id) + ".jpg"
    return name

In [ ]:
def load_feature_cache(feature_names):

    for feature_name in feature_names:
        print('processing:', feature_name)
        feat_root = os.path.join(root + '/models/ABAW6', feature_name)
        save_root = feat_root+'.pkl'
        if os.path.exists(save_root):
            continue
        filenames = os.listdir(feat_root)[:]
        feat = {}
        for fname in tqdm(filenames):
            vname = fname.split('.')[0]
            if filenames[0].endswith('.npy'):
                fea = np.load(os.path.join(feat_root, fname), allow_pickle=True).tolist()
            elif filenames[0].endswith('.pkl'):
                with open(os.path.join(feat_root, fname), 'rb') as f:
                    fea = pickle.load(f)
            feat[vname] = fea
        with open(save_root, 'wb') as f:
            pickle.dump(feat, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
feature_names = [
        'visualfeat_enet_b2_8_best_cropped',
        'visualfeat_enet_b2_8_best_cropped_aligned',
        'audiofeat_vggish',
        'audiofeat_wav2vec2'
        ]

In [ ]:
class ABAW_dataset(Dataset):
    def __init__(self, root, split, typ, task, feature_v, feature_a):
        self.root = root
        self.split = split
        self.typ = typ
        self.task = task
        self.anno_path = os.path.join(self.root,f'data/Annotations/{self.task}/{self.split}')
        self.feature_v, self.feature_a = feature_v, feature_a
        self.feature = [self.feature_v, self.feature_a]
        with open(os.path.join(root, f'data/Annotations/{self.task}/{self.typ}.txt'), 'r') as f:
                self.vidnames = f.read().splitlines()
        self.feature_dims = 0
        self.data = {}
        self.data[self.task] = {}
        self.iname = []
        for feature_name in self.feature:
            if 'visual' in feature_name:
                self.data = self.load_feature_v(feature_name)
            elif 'audio' in feature_name:
                self.data = self.load_feature_a(feature_name)

    def get_names(vid, id):
            name = ""
            if id>=0 and id<10:
                name = f"{vid}/0000" + str(id) + ".jpg"
            elif id>=10 and id<100:
                name = f"{vid}/000" + str(id) + ".jpg"
            elif id>=100 and id<1000:
                name = f"{vid}/00" + str(id) + ".jpg"
            elif id>=1000 and id<10000:
                name = f"{vid}/0" + str(id) + ".jpg"
            else:
                name = f"{vid}/" + str(id) + ".jpg"
            return name

    def load_feature_v(self, feature_v):
            print(f'loading visual feature: {feature_v}')
            feat_root = os.path.join(root + '/models/ABAW6', feature_v)
            filenames = os.listdir(feat_root)[:]
            for vname in tqdm(self.vidnames):
                    feature = np.load(os.path.join(feat_root, f'{vname}.npy'), allow_pickle=True).tolist()
                    with open(os.path.join(self.anno_path, f'{vname}.txt')) as f:
                        labels = f.read().splitlines()
                    self.data[self.task][vname] = {}

                    for imgname, val in feature.items():
                        for i,line in enumerate(labels):
                            if i > 0:
                                imname = get_names(vname, i)
                                if imname == imgname:
                                    if self.task == 'AU_Detection_Challenge':
                                        splitted_line=line.split(',')
                                        aus = list(map(int,splitted_line))
                                        if min(aus) >= 0:
                                            labs = torch.tensor(aus)
                                            self.data[self.task][vname][imgname] = {f'{feature_v}': val, 'label': labs}
                                            self.iname.append(imname)
                                    elif self.task == 'VA_Estimation_Challenge':
                                        splitted_line=line.split(',')
                                        valence=float(splitted_line[0])
                                        arousal=float(splitted_line[1])
                                        if valence >= -1 and arousal >= -1:
                                            labs = torch.tensor([valence, arousal])
                                            self.data[self.task][vname][imgname] = {f'{feature_v}': val, 'label': labs}
                                            self.iname.append(imname)
                                    elif self.task == 'EXPR_Recognition_Challenge':
                                        exp = int(line)
                                        if exp >= 0:
                                            labs = torch.tensor(exp)
                                            self.data[self.task][vname][imgname] = {f'{feature_v}': val, 'label': labs}
                                            self.iname.append(imname)
                    self.feature_dims += len(self.data[self.task][vname])
            return self.data

    def load_feature_a(self, feature_a):
            print(f'loading audio feature: {feature_a}')
            feat_root = os.path.join(root + '/models/ABAW6', feature_a)
            filenames = os.listdir(feat_root)[:]
            for vname in tqdm(self.vidnames):
                    feature = np.load(os.path.join(feat_root, f'{vname}.npy'), allow_pickle=True).tolist()
                    for imgname, val in feature.items():
                        if imgname in self.data[self.task][vname]:
                            self.data[self.task][vname][imgname].update({f'{feature_a}': val})

                    for img, value in list(self.data[self.task][vname].items()):
                        if len(value) < 3:
                            self.data[self.task][vname].pop(img)
            return self.data

    def __getitem__(self, index):
            frame = self.iname[index]
            vname = frame.split('/')[0]
            data = self.data[self.task][vname][frame]
            data['frame'] = frame
            data['vid'] = vname
            data['label'] = self.data[self.task][vname][frame]['label']
            return data

    def __len__(self):
            return self.feature_dims

In [ ]:
class ABAW_dataset1(Dataset):
    def __init__(self, data, iname, dims, task):
        self.data = data
        self.iname = iname
        self.task = task
        self.feature_dims = dims
    def __getitem__(self, index):
        frame = self.iname[index]
        vname = frame.split('/')[0]
        data = self.data[self.task][vname][frame]
        data['frame'] = frame
        data['vid'] = vname
        data['label'] = self.data[self.task][vname][frame]['label']
        return data

    def __len__(self):
            return self.feature_dims

# Convert

## Video to Audio

In [ ]:
def vid2aud(video_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for file_name in tqdm(os.listdir(video_folder)):
        video_path = os.path.join(video_folder, file_name)
        audio_file_name = os.path.splitext(file_name)[0] + '.wav'
        audio_file_path = os.path.join(output_folder, audio_file_name)
        if os.path.exists(audio_file_path):
            continue
        if os.path.isfile(video_path) and file_name.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            video_clip = VideoFileClip(video_path)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(audio_file_path)
            audio_clip.close()
            video_clip.close()

In [ ]:
for i in ['batch1','batch2','new_vids']:
    video_folder= os.path.join(root,'data/video', i)
    print(f'\nProcessing {video_folder}')
    output_folder= os.path.join(root, 'data/audio')
    vid2aud(video_folder, output_folder)


Processing /content/drive/MyDrive/MSc/Thesis/data/video/batch1


100%|██████████| 475/475 [00:02<00:00, 192.89it/s]



Processing /content/drive/MyDrive/MSc/Thesis/data/video/batch2


100%|██████████| 73/73 [00:00<00:00, 2263.92it/s]



Processing /content/drive/MyDrive/MSc/Thesis/data/video/new_vids


100%|██████████| 50/50 [00:00<00:00, 540.31it/s]


## Audio to Text

In [ ]:
class Fuxi():
    def __init__(self, max_duration=50):
        self.appkey = 'phr-fuxi'
        self.appsecret = '74c72dee-bf9a-4de2-8c1f-96be1a1ecabd'
        self.url = 'http://api-test.vop.netease.com/phone_rec'
        self.max_duration = max_duration

    def run(self, wav_path, lang='en', type="wav",max_duration=50):
        words = []

        audio = AudioSegment.from_file(wav_path, format='mp3')
        duration_ms = len(audio)
        if duration_ms < 500:
            return ''
        chunk_length_ms = max_duration * 1000  # 60s
        text = ''
        long_vid = []
        for i in range(0, duration_ms,chunk_length_ms):
            new_audio = audio[i:min(i+chunk_length_ms,duration_ms)]
            byte_io = BytesIO()
            new_audio.export(byte_io, format="wav")
            speech = byte_io.getvalue()

            curtime = str(int(time.time()))
            hl = hashlib.md5()
            hl.update((self.appkey + curtime).encode(encoding='utf-8'))
            sign = hmac.new(self.appsecret.encode('utf-8'),
                            hl.hexdigest().encode('utf-8'), hashlib.sha1).digest()
            checksum = base64.b64encode(sign)
            params = {'appkey': self.appkey, 'lan': lang}
            headers = {
                'curtime': curtime,
                'checksum': checksum,
                'content-type': 'audio/wav',
                'cuid': 'fuxi-avatarlib'
            }
            response = requests.post(self.url,
                                        params=params,
                                        headers=headers,
                                        data=speech)
            r = response.json()
            if r['ret_code'] != 1:
                #error = r['ret_msg']
                #raise RuntimeError('Hangyan Rec Error: ' + error+f'[{wav_path}]')
                long_vid.append(wav_path)
                pass
            aligned_text = r['result']
            text += ' '.join([w['word'] for w in aligned_text if w['word'] != 'sil'])
        return text

    def split_wave(self, wav_path, save_root, max_duration=50):
        wav_name = wav_path.split('/')[-1].replace('.wav', '')
        fin = wave.open(wav_path, 'rb')
        fs_orig = fin.getframerate()
        audio_length = fin.getnframes() * (1/fs_orig)
        fin.close()

        audio = AudioSegment.from_wav(wav_path)
        n = int(audio_length//max_duration + 1)
        for i in range(n):
            new_audio = audio[i*max_duration*1000:(i+1)*max_duration*1000]
            new_audio.export(os.path.join(save_root, wav_name+f'_{i}.wav'), format='wav')
        return n

    def read_wave(self, wav_path, max_duration=60):
        with open(wav_path, 'rb') as f:
            wav_data = f.read()
        return wav_data

In [ ]:
def aud2text():

    speech2text = Fuxi()
    print('loading model')
    root_wav = os.path.join(root, 'data/audio')
    wav_files = os.listdir(root_wav)[::-1]

    print(f'processing {root_wav}')
    save_folder = os.path.join(root, f'models/ABAW6/aud2text')
    os.makedirs(save_folder, exist_ok=True)

    res_dict = {}
    for audio_name in tqdm(wav_files):
        audio_file = os.path.join(root_wav, audio_name)
        save_path = os.path.join(save_folder, audio_name.split('.')[0]+ '.json')
        if os.path.exists(save_path):
            with open(save_path, 'r') as f:
                text = json.load(f)
            continue
        else:
            text = speech2text.run(audio_file)
            with open(save_path, 'w') as f:
                json.dump(text, f)
        res_dict[audio_name] = text.strip()

    with open(os.path.join(save_folder,'alltext.json'), 'w') as f:
        json.dump(res_dict, f, indent=4)

In [ ]:
aud2text()

loading model
processing /content/drive/MyDrive/MSc/Thesis/data/audio


100%|██████████| 173/173 [52:28<00:00, 18.20s/it]


# Metrics

### Compute loss

In [ ]:
def compute_EXP_loss(pred, label, weights):
    cri_exp = nn.CrossEntropyLoss(weights)
    cls_loss = cri_exp(pred, label)
    return cls_loss

In [ ]:
def compute_AU_loss(pred, label, class_weights):
    class_weights = torch.from_numpy(class_weights).to(device)
    bce = F.binary_cross_entropy_with_logits(pred, label.float(), reduction='none')
    weights = (class_weights[:, 0]**(1 - label)) * (class_weights[:, 1]**label)
    weighted_bce = bce * weights
    loss = torch.mean(weighted_bce)
    return loss

In [ ]:
def CCC_loss(x, y):
    x, y = x.view(-1), y.view(-1)
    vx = x - torch.mean(x)
    vy = y - torch.mean(y)
    rho =  torch.sum(vx * vy) / (torch.sqrt(torch.sum(torch.pow(vx, 2))) * torch.sqrt(torch.sum(torch.pow(vy, 2)))+1e-8)
    x_m, y_m = torch.mean(x), torch.mean(y)
    x_s, y_s = torch.std(x), torch.std(y)
    ccc = 2*rho*x_s*y_s/(torch.pow(x_s, 2) + torch.pow(y_s, 2) + torch.pow(x_m - y_m, 2))
    return 1-ccc

In [ ]:
def compute_VA_loss(Vout,Aout,label):
    ccc_loss = CCC_loss(Vout[:,0],label[:,0]) + CCC_loss(Aout[:,0],label[:,1])
    mse_loss = nn.MSELoss()(Vout,label[:,0]) + nn.MSELoss()(Aout,label[:,1])
    return mse_loss,ccc_loss

### Compute F1 score

In [ ]:
def compute_EXP_F1(pred, target):
    pred_labels = np.argmax(pred, axis=1)
    target_labels = np.argmax(target, axis=1)
    macro_f1 = f1_score(target_labels,pred_labels,average='macro')
    acc = accuracy_score(target_labels, pred_labels)
    return macro_f1, acc

In [ ]:
def f1s_max_AU(label, pred, thresh, i=0):
    pred = np.array(pred)
    label = np.array(label)
    label = label[:,i]
    pred = pred[:,i]
    acc = []
    F1 = []
    for i in thresh:
        new_pred = ((pred >= i) * 1).flatten()
        acc.append(accuracy_score(label.flatten(), new_pred))
        F1.append(f1_score(label.flatten(), new_pred))

    F1_MAX = max(F1)
    if F1_MAX < 0 or math.isnan(F1_MAX):
        F1_MAX = 0
        F1_THRESH = 0
        accuracy = 0
    else:
        idx_thresh = np.argmax(F1)
        F1_THRESH = thresh[idx_thresh]
        accuracy = acc[idx_thresh]
    return F1, F1_MAX, F1_THRESH, accuracy

In [ ]:
def compute_AU_F1(pred,label,thresh=np.arange(0.1,1,0.1)):
    F1s = []
    F1t = []
    acc = []
    for i in range(12):
        F1, F1_MAX, F1_THRESH, accuracy = f1s_max_AU(label,pred,thresh,i)
        F1s.append(F1_MAX)
        F1t.append(F1_THRESH)
        acc.append(accuracy)
    acc = [round(a,3) for a in acc]
    return np.mean(F1s),np.mean(F1t),acc, F1t

### Concordance Correlation Coefficient

In [ ]:
def CCC_score(x, y):
    x = np.array(x)
    y = np.array(y)
    vx = x - np.mean(x)
    vy = y - np.mean(y)
    rho = np.sum(vx * vy) / (np.sqrt(np.sum(vx**2)) * np.sqrt(np.sum(vy**2)))
    x_m = np.mean(x)
    y_m = np.mean(y)
    x_s = np.std(x)
    y_s = np.std(y)
    ccc = 2*rho*x_s*y_s/(x_s**2 + y_s**2 + (x_m - y_m)**2)
    return ccc

In [ ]:
def compute_VA_CCC(x,y):
    x = np.array(x)
    y = np.array(y)
    x[x>1] = 1
    x[x<-1] = -1
    ccc1 = CCC_score(x[:,0],y[:,0])
    ccc2 = CCC_score(x[:,1],y[:,1])

    return ccc1,ccc2

# Smooth utils

In [ ]:
def smooth_prediction(img, predict):
    cur_ind = 0
    preds_proba = []
    if img:
        for i in range(img[-1]):
            if img[cur_ind] - 1 == i:
                preds_proba.append(predict[cur_ind])
                cur_ind += 1
            else:
                if cur_ind == 0:
                    preds_proba.append(predict[cur_ind])
                else:
                    w = (i - img[cur_ind - 1] + 1) / (img[cur_ind] - img[cur_ind - 1])
                    pred = w * predict[cur_ind - 1] + (1 - w) * predict[cur_ind]
                    preds_proba.append(pred)
        try:
            preds_proba = np.array([p.cpu().detach().numpy() for p in preds_proba])
        except:
            preds_proba = np.array(preds_proba)
        return preds_proba

In [ ]:
def slide_window(preds_proba, i, delta, typ):
    i1 = max(i - delta, 0)
    if typ == 'mean':
        proba = np.mean(preds_proba[i1:i+delta+1], axis=0)
    elif typ == 'median':
        proba = np.median(preds_proba[i1:i+delta+1], axis=0)
    else:
        proba = np.median(preds_proba[i1:i+delta+1:int(typ)], axis=0)
    return np.argmax(proba), proba

# Challenges

In [ ]:
task = ['EXPR_Recognition_Challenge','AU_Detection_Challenge','VA_Estimation_Challenge']
split = ['Train_Set', 'Validation_Set']
typ = ['Train','Val','Test']
vis_typ = ['cropped_aligned', 'cropped', 'cropped_aligned_b0']
visual_feat = 'visualfeat_enet_b2_8_best'
visual_feat_1 = 'visualfeat_enet_b0_8_va_mtl'
audio_feat = ['audiofeat_wav2vec2','audiofeat_vggish','nope']
vis_aud = ['visual_wav2vec2','visual_vggish','visual']
batch_size = 32
model_type = ['fusion', 'mlp']

## EXPR Recognition Challenge

### Loading data

In [ ]:
# Cropped_aligned images
vis = vis_typ[0]

#### Effnet + wav2vec2

In [ ]:
auft = audio_feat[0]
viau = vis_aud[0]

#### Effnet + vggish

In [ ]:
auft = audio_feat[1]
viau = vis_aud[1]

#### Effnet

In [ ]:
auft = audio_feat[2]
viau = vis_aud[2]

#### Train

In [ ]:
with open(os.path.join(root,f'models/ABAW6/{vis}/EXPR/{task[0]}_{typ[0]}_{viau}.pkl'), 'rb') as f:
    data1 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[0]
with open(os.path.join(root, f'data/Annotations/{task[0]}/{typ[0]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data1[task1][vname])
    for img in data1[task1][vname].keys():
        iname.append(img)

In [ ]:
dataset = ABAW_dataset1(data1, iname, dims, task1)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0, drop_last=True)

#### Val

In [ ]:
with open(os.path.join(root,f'models/ABAW6/{vis}/EXPR/{task[0]}_{typ[1]}_{viau}.pkl'), 'rb') as f:
    data2 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[0]
with open(os.path.join(root, f'data/Annotations/{task[0]}/{typ[1]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data2[task1][vname])
    for img in data2[task1][vname].keys():
        iname.append(img)

In [ ]:
dataset = ABAW_dataset1(data2, iname, dims, task1)
val_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, drop_last=True)

#### Test

In [ ]:
with open(os.path.join(root,f'models/ABAW6/{vis}/EXPR/{task[0]}_{typ[2]}_{viau}.pkl'), 'rb') as f:
    data3 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[0]
with open(os.path.join(root, f'data/Annotations/{task[0]}/{typ[2]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data3[task1][vname])
    for img in data3[task1][vname].keys():
        iname.append(img)

100%|██████████| 70/70 [00:00<00:00, 1553.79it/s]


In [ ]:
dataset = ABAW_dataset1(data3, iname, dims, task1)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, drop_last=False)

#### Another way (Effnet + wav2vec2)

In [ ]:
print(f'{task[0]}')
print(f'loading {typ[0]}')
dataset = ABAW_dataset(root, split[0], typ[0], task[0], feature_v=visual_feat, feature_a=audio_feat)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
torch.save(loader, os.path.join(root,f'models/ABAW6/{task[0]}_{typ[0]}.pth'))

In [ ]:
print(f'{task[0]}')
print(f'loading {typ[1]}')
dataset = ABAW_dataset(root, split[0], typ[1], task[0], feature_v=visual_feat, feature_a=audio_feat)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)
torch.save(loader, os.path.join(root,f'models/ABAW6/{task[0]}_{typ[1]}.pth'))

In [ ]:
print(f'{task[0]}')
print(f'loading {typ[2]}')
dataset = ABAW_dataset(root, split[1], typ[2], task[0], feature_v=visual_feat, feature_a=audio_feat)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2,drop_last=True)
torch.save(loader, os.path.join(root,f'models/ABAW6/{task[0]}_{typ[2]}.pth'))

In [ ]:
train_loader = torch.load(os.path.join(root,f'models/ABAW6/EXPR/{task[0]}_{typ[0]}.pth'))

In [ ]:
val_loader = torch.load(os.path.join(root,f'models/ABAW6/EXPR/{task[0]}_{typ[1]}.pth'))

In [ ]:
test_loader = torch.load(os.path.join(root,f'models/ABAW6/EXPR/{task[0]}_{typ[2]}.pth'))

### Modeling

#### Transformer encoder

In [ ]:
class EXP_fusion(nn.Module):
    def __init__(self, batchsize = batch_size, audio_ft = auft, hidden_size = [512, 128, batch_size]):
        super(EXP_fusion, self).__init__()
        self.batchsize = batchsize
        if audio_ft == 'audiofeat_wav2vec2':
            self.concat_dim = 2176    #1408+768
        elif audio_ft == 'audiofeat_vggish':
            self.concat_dim = 1536    #1408+128
        elif audio_ft == 'nope':
            self.concat_dim = 1408    #visual only
        self.hidden_size = hidden_size
        self.feat_fc = nn.Conv1d(self.concat_dim, hidden_size[0], 1, padding=0)
        self.activ = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(p=0.3)
        self.conv1 = nn.Conv1d(hidden_size[0], hidden_size[1], 1, padding=0)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size[1], nhead=4, dim_feedforward=hidden_size[1], dropout=0.3)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)
        self.head = nn.Sequential(
                nn.Linear(hidden_size[1], hidden_size[2]),
                nn.BatchNorm1d(hidden_size[2]),
                nn.Dropout(p=0.3),
                nn.Linear(hidden_size[2], 8))

    def forward(self, vis_feat, aud_feat):
        if aud_feat == None:
            feat = vis_feat
        else:
            inputs = [vis_feat]
            inputs.append(aud_feat)
            feat = torch.cat(inputs,dim=1)
        feat = torch.transpose(feat,0,1)
        feat = self.feat_fc(feat)
        feat = self.activ(feat)
        out = self.conv1(feat)
        out = torch.transpose(out,0,1)
        out = self.transformer_encoder(out)
        out = self.head(out)

        return out, torch.softmax(out, dim = 1)

In [ ]:
EXP_model = EXP_fusion().to(device)
EXP_model

EXP_fusion(
  (feat_fc): Conv1d(1536, 512, kernel_size=(1,), stride=(1,))
  (activ): LeakyReLU(negative_slope=0.1)
  (dropout): Dropout(p=0.3, inplace=False)
  (conv1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=128, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
    )
  )
  (head): Sequential(
    (0): Linear(in_features=128, out_feature

#### MLP

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, audio_ft = auft, num_classes=8):
        super(MLPModel, self).__init__()
        if audio_ft == 'audiofeat_wav2vec2':
            self.concat_dim = 2176    #1408+768
        elif audio_ft == 'audiofeat_vggish':
            self.concat_dim = 1536    #1408+128
        elif audio_ft == 'nope':
            self.concat_dim = 1408    #visual only
        self.activ = nn.ReLU()
        self.fc1 = nn.Linear(self.concat_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, vis_feat, aud_feat):
        if aud_feat == None:
            feat = vis_feat
        else:
            inputs = [vis_feat]
            inputs.append(aud_feat)
            feat = torch.cat(inputs, dim=1)
        feat = self.fc1(feat)
        feat = self.activ(feat)
        out = self.fc2(feat)
        return out, torch.softmax(out, dim=1)

In [ ]:
mlp_model = MLPModel().to(device)
mlp_model

MLPModel(
  (activ): ReLU()
  (fc1): Linear(in_features=2176, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=8, bias=True)
)

#### Utils

In [ ]:
y_train = []
iterator = iter(train_loader)
i = 0
while True:
    try:
        EXPR = next(iterator)
        y_train.extend(EXPR['label'].numpy())
    except:
        break

In [ ]:
class_weights=compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)

weights=torch.tensor(class_weights,dtype=torch.float).to(device)
print(f'{weights = }')

In [ ]:
weights = [0.4260, 4.1334, 5.9334, 6.8653, 0.7257, 0.9578, 2.0518, 0.4572]
weights = torch.tensor(weights).to(device)

In [ ]:
def one_hot_transfer(label, class_num):
    one_hot = torch.eye(class_num)
    one_hot = one_hot.to(device)
    return one_hot[label]

In [ ]:
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, EXP_model.parameters()), lr=0.00001, betas=(0.9, 0.999), weight_decay=0.00005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=1e-5)

In [ ]:
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, mlp_model.parameters()), lr=0.00001, betas=(0.9, 0.999), weight_decay=0.00005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=1e-5)

### Training

In [ ]:
def train(model, mod_type, train_loader, val_loader, epoch, batch_size, optim, au_feat, weight, vi_au):

    model.train(True)
    model.eval()
    best_loss = float('inf')
    f1best, accbest = 0, 0
    loss_value = []
    loss_train = []
    loss_val = []
    all_preds = []
    all_targets = []

    for e in range(epoch):
        print(f'Epoch: {e+1}')
        iterator = iter(train_loader)
        while True:
            try:
                EXPR = next(iterator)
                if au_feat == 'nope':
                    vis_feat, y = EXPR[visual_feat], EXPR['label']
                    vis_feat, y = vis_feat.to(device), y.to(device)
                    aud_feat = None
                else:
                    vis_feat, aud_feat, y = EXPR[visual_feat], EXPR[au_feat], EXPR['label']
                    vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
                y_onehot = one_hot_transfer(y, 8).to(device)
                model.zero_grad()
                pred, exp_pred = model(vis_feat, aud_feat)
                loss = compute_EXP_loss(pred, y_onehot, weight)
                loss.backward()
                optim.step()
                loss_value.append(loss.item())
                all_preds.extend(exp_pred.cpu().tolist())
                all_targets.extend(y_onehot.cpu().tolist())
            except:
                break
        avg_loss = round(np.mean(loss_value),3)
        loss_train.append(avg_loss)
        f1_scores, accuracy = compute_EXP_F1(all_preds, all_targets)
        print(f'Train Loss: {avg_loss}, Accuracy: {round(accuracy,3)}')

        val_loss, f1s, acc = evaluate_model(model, val_loader, au_feat, weight)
        loss_val.append(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_{mod_type}_{vi_au}.pth'))
            f1best = f1s
            accbest = acc

        print(f'Validation Loss: {val_loss}, Accuracy: {acc}')
        scheduler.step(val_loss)
    return loss_train, loss_val, best_loss, f1best, accbest

In [ ]:
def evaluate_model(model, data_loader, au_feat, weight):
    model.eval()
    total_loss = []
    all_preds = []
    all_targets = []
    with torch.no_grad():
        iterator = iter(data_loader)
        for EXPR in iterator:
            if au_feat == 'nope':
                vis_feat, y = EXPR[visual_feat], EXPR['label']
                vis_feat, y = vis_feat.to(device), y.to(device)
                aud_feat = None
            else:
                vis_feat, aud_feat, y = EXPR[visual_feat], EXPR[au_feat], EXPR['label']
                vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
            y_onehot = one_hot_transfer(y, 8).to(device)
            pred, exp_pred = model(vis_feat, aud_feat)
            loss = compute_EXP_loss(pred, y_onehot, weight)
            total_loss.append(loss.item())
            all_preds.extend(exp_pred.cpu().tolist())
            all_targets.extend(y_onehot.cpu().tolist())

    f1_scores, acc = compute_EXP_F1(all_preds, all_targets)
    return round(np.mean(total_loss),3), round(f1_scores,3), round(acc,3)

#### Cropped_aligned images

##### Effnet + Wav2vec2

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(EXP_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.68, Accuracy: 0.737
Validation Loss: 1.494, Accuracy: 0.468
Epoch: 2
Train Loss: 0.497, Accuracy: 0.801
Validation Loss: 1.709, Accuracy: 0.459
Epoch: 3
Train Loss: 0.407, Accuracy: 0.833
Validation Loss: 1.757, Accuracy: 0.494
Epoch: 4
Train Loss: 0.351, Accuracy: 0.853
Validation Loss: 1.883, Accuracy: 0.48
Epoch: 5
Train Loss: 0.31, Accuracy: 0.868
Validation Loss: 2.133, Accuracy: 0.484
Epoch: 6
Train Loss: 0.28, Accuracy: 0.879
Validation Loss: 2.186, Accuracy: 0.497
Epoch: 7
Train Loss: 0.256, Accuracy: 0.888
Validation Loss: 2.404, Accuracy: 0.489
Epoch: 8
Train Loss: 0.236, Accuracy: 0.896
Validation Loss: 2.459, Accuracy: 0.471
Epoch: 9
Train Loss: 0.219, Accuracy: 0.902
Validation Loss: 2.515, Accuracy: 0.507
Epoch: 10
Train Loss: 0.205, Accuracy: 0.908
Validation Loss: 2.705, Accuracy: 0.502
28min 25s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_fusion_{viau}.pth'))
    EXP_model = EXP_fusion().to(device)
    EXP_model.load_state_dict(EXP_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(EXP_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.282, accuracy: 0.468


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 1.274, Accuracy: 0.539
Validation Loss: 1.278, Accuracy: 0.443
Epoch: 2
Train Loss: 1.071, Accuracy: 0.602
Validation Loss: 1.366, Accuracy: 0.447
Epoch: 3
Train Loss: 0.96, Accuracy: 0.64
Validation Loss: 1.414, Accuracy: 0.452
Epoch: 4
Train Loss: 0.885, Accuracy: 0.666
Validation Loss: 1.453, Accuracy: 0.456
Epoch: 5
Train Loss: 0.829, Accuracy: 0.685
Validation Loss: 1.481, Accuracy: 0.456
Epoch: 6
Train Loss: 0.786, Accuracy: 0.701
Validation Loss: 1.524, Accuracy: 0.456
Epoch: 7
Train Loss: 0.751, Accuracy: 0.713
Validation Loss: 1.546, Accuracy: 0.452
Epoch: 8
Train Loss: 0.721, Accuracy: 0.724
Validation Loss: 1.563, Accuracy: 0.456
Epoch: 9
Train Loss: 0.695, Accuracy: 0.733
Validation Loss: 1.586, Accuracy: 0.451
Epoch: 10
Train Loss: 0.673, Accuracy: 0.741
Validation Loss: 1.604, Accuracy: 0.45
7min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.29, accuracy: 0.452


##### Effnet + Vggish

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(EXP_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.668, Accuracy: 0.74
Validation Loss: 1.387, Accuracy: 0.492
Epoch: 2
Train Loss: 0.471, Accuracy: 0.805
Validation Loss: 1.575, Accuracy: 0.481
Epoch: 3
Train Loss: 0.373, Accuracy: 0.84
Validation Loss: 1.922, Accuracy: 0.473
Epoch: 4
Train Loss: 0.311, Accuracy: 0.864
Validation Loss: 2.178, Accuracy: 0.495
Epoch: 5
Train Loss: 0.268, Accuracy: 0.881
Validation Loss: 2.274, Accuracy: 0.467
Epoch: 6
Train Loss: 0.236, Accuracy: 0.894
Validation Loss: 2.43, Accuracy: 0.495
Epoch: 7
Train Loss: 0.211, Accuracy: 0.905
Validation Loss: 2.607, Accuracy: 0.475
Epoch: 8
Train Loss: 0.191, Accuracy: 0.913
Validation Loss: 2.66, Accuracy: 0.488
Epoch: 9
Train Loss: 0.175, Accuracy: 0.92
Validation Loss: 2.806, Accuracy: 0.477
Epoch: 10
Train Loss: 0.161, Accuracy: 0.926
Validation Loss: 2.994, Accuracy: 0.47
29min 23s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_fusion_{viau}.pth'))
    EXP_model = EXP_fusion().to(device)
    EXP_model.load_state_dict(EXP_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(EXP_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.286, accuracy: 0.492


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(mlp_model,model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 1.417, Accuracy: 0.495
Validation Loss: 1.21, Accuracy: 0.416
Epoch: 2
Train Loss: 1.232, Accuracy: 0.548
Validation Loss: 1.213, Accuracy: 0.454
Epoch: 3
Train Loss: 1.121, Accuracy: 0.585
Validation Loss: 1.218, Accuracy: 0.489
Epoch: 4
Train Loss: 1.043, Accuracy: 0.613
Validation Loss: 1.23, Accuracy: 0.493
Epoch: 5
Train Loss: 0.985, Accuracy: 0.633
Validation Loss: 1.229, Accuracy: 0.501
Epoch: 6
Train Loss: 0.938, Accuracy: 0.649
Validation Loss: 1.24, Accuracy: 0.499
Epoch: 7
Train Loss: 0.9, Accuracy: 0.663
Validation Loss: 1.237, Accuracy: 0.506
Epoch: 8
Train Loss: 0.868, Accuracy: 0.674
Validation Loss: 1.252, Accuracy: 0.496
Epoch: 9
Train Loss: 0.84, Accuracy: 0.683
Validation Loss: 1.237, Accuracy: 0.496
Epoch: 10
Train Loss: 0.816, Accuracy: 0.692
Validation Loss: 1.26, Accuracy: 0.499
7min 35s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.309, accuracy: 0.501


##### Effnet

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(EXP_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.7, Accuracy: 0.729
Validation Loss: 1.413, Accuracy: 0.46
Epoch: 2
Train Loss: 0.529, Accuracy: 0.786
Validation Loss: 1.596, Accuracy: 0.445
Epoch: 3
Train Loss: 0.441, Accuracy: 0.816
Validation Loss: 1.822, Accuracy: 0.478
Epoch: 4
Train Loss: 0.385, Accuracy: 0.837
Validation Loss: 1.869, Accuracy: 0.472
Epoch: 5
Train Loss: 0.345, Accuracy: 0.852
Validation Loss: 2.031, Accuracy: 0.466
Epoch: 6
Train Loss: 0.314, Accuracy: 0.863
Validation Loss: 2.083, Accuracy: 0.434
Epoch: 7
Train Loss: 0.289, Accuracy: 0.873
Validation Loss: 2.298, Accuracy: 0.472
Epoch: 8
Train Loss: 0.268, Accuracy: 0.881
Validation Loss: 2.361, Accuracy: 0.459
Epoch: 9
Train Loss: 0.251, Accuracy: 0.888
Validation Loss: 2.384, Accuracy: 0.455
Epoch: 10
Train Loss: 0.236, Accuracy: 0.894
Validation Loss: 2.432, Accuracy: 0.464
27min 18s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_fusion_{viau}.pth'))
    EXP_model = EXP_fusion().to(device)
    EXP_model.load_state_dict(EXP_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(EXP_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.262, accuracy: 0.478


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 1.477, Accuracy: 0.467
Validation Loss: 1.214, Accuracy: 0.388
Epoch: 2
Train Loss: 1.325, Accuracy: 0.505
Validation Loss: 1.222, Accuracy: 0.402
Epoch: 3
Train Loss: 1.232, Accuracy: 0.535
Validation Loss: 1.235, Accuracy: 0.412
Epoch: 4
Train Loss: 1.164, Accuracy: 0.558
Validation Loss: 1.25, Accuracy: 0.428
Epoch: 5
Train Loss: 1.11, Accuracy: 0.577
Validation Loss: 1.261, Accuracy: 0.43
Epoch: 6
Train Loss: 1.065, Accuracy: 0.593
Validation Loss: 1.281, Accuracy: 0.429
Epoch: 7
Train Loss: 1.027, Accuracy: 0.606
Validation Loss: 1.278, Accuracy: 0.441
Epoch: 8
Train Loss: 0.995, Accuracy: 0.618
Validation Loss: 1.291, Accuracy: 0.436
Epoch: 9
Train Loss: 0.967, Accuracy: 0.628
Validation Loss: 1.301, Accuracy: 0.439
Epoch: 10
Train Loss: 0.942, Accuracy: 0.637
Validation Loss: 1.315, Accuracy: 0.436
6min 29s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.275, accuracy: 0.441


#### Cropped images

##### Effnet + Wav2vec2

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(EXP_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.742, Accuracy: 0.71
Validation Loss: 1.544, Accuracy: 0.483
Epoch: 2
Train Loss: 0.559, Accuracy: 0.775
Validation Loss: 1.756, Accuracy: 0.477
Epoch: 3
Train Loss: 0.468, Accuracy: 0.807
Validation Loss: 1.852, Accuracy: 0.514
Epoch: 4
Train Loss: 0.409, Accuracy: 0.828
Validation Loss: 2.096, Accuracy: 0.512
Epoch: 5
Train Loss: 0.367, Accuracy: 0.843
Validation Loss: 2.13, Accuracy: 0.509
Epoch: 6
Train Loss: 0.334, Accuracy: 0.855
Validation Loss: 2.323, Accuracy: 0.489
Epoch: 7
Train Loss: 0.308, Accuracy: 0.865
Validation Loss: 2.354, Accuracy: 0.506
Epoch: 8
Train Loss: 0.286, Accuracy: 0.873
Validation Loss: 2.506, Accuracy: 0.511
Epoch: 9
Train Loss: 0.268, Accuracy: 0.88
Validation Loss: 2.579, Accuracy: 0.518
Epoch: 10
Train Loss: 0.252, Accuracy: 0.886
Validation Loss: 2.697, Accuracy: 0.503
28min 16s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_fusion_{viau}.pth'))
    EXP_model = EXP_fusion().to(device)
    EXP_model.load_state_dict(EXP_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(EXP_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.269, accuracy: 0.518


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 1.372, Accuracy: 0.495
Validation Loss: 1.248, Accuracy: 0.425
Epoch: 2
Train Loss: 1.168, Accuracy: 0.56
Validation Loss: 1.331, Accuracy: 0.452
Epoch: 3
Train Loss: 1.052, Accuracy: 0.6
Validation Loss: 1.379, Accuracy: 0.472
Epoch: 4
Train Loss: 0.973, Accuracy: 0.628
Validation Loss: 1.454, Accuracy: 0.467
Epoch: 5
Train Loss: 0.915, Accuracy: 0.649
Validation Loss: 1.492, Accuracy: 0.465
Epoch: 6
Train Loss: 0.87, Accuracy: 0.665
Validation Loss: 1.535, Accuracy: 0.467
Epoch: 7
Train Loss: 0.833, Accuracy: 0.678
Validation Loss: 1.575, Accuracy: 0.464
Epoch: 8
Train Loss: 0.803, Accuracy: 0.69
Validation Loss: 1.607, Accuracy: 0.463
Epoch: 9
Train Loss: 0.776, Accuracy: 0.699
Validation Loss: 1.635, Accuracy: 0.467
Epoch: 10
Train Loss: 0.754, Accuracy: 0.707
Validation Loss: 1.656, Accuracy: 0.472
7min 25s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.308, accuracy: 0.472


##### Effnet + Vggish

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(EXP_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.779, Accuracy: 0.695
Validation Loss: 1.356, Accuracy: 0.511
Epoch: 2
Train Loss: 0.586, Accuracy: 0.759
Validation Loss: 1.504, Accuracy: 0.512
Epoch: 3
Train Loss: 0.484, Accuracy: 0.794
Validation Loss: 1.683, Accuracy: 0.509
Epoch: 4
Train Loss: 0.416, Accuracy: 0.818
Validation Loss: 1.844, Accuracy: 0.507
Epoch: 5
Train Loss: 0.366, Accuracy: 0.836
Validation Loss: 2.024, Accuracy: 0.501
Epoch: 6
Train Loss: 0.329, Accuracy: 0.85
Validation Loss: 2.264, Accuracy: 0.495
Epoch: 7
Train Loss: 0.298, Accuracy: 0.862
Validation Loss: 2.425, Accuracy: 0.493
Epoch: 8
Train Loss: 0.273, Accuracy: 0.872
Validation Loss: 2.425, Accuracy: 0.5
Epoch: 9
Train Loss: 0.252, Accuracy: 0.881
Validation Loss: 2.808, Accuracy: 0.493
Epoch: 10
Train Loss: 0.235, Accuracy: 0.889
Validation Loss: 2.79, Accuracy: 0.497
28min 26s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_fusion_{viau}.pth'))
    EXP_model = EXP_fusion().to(device)
    EXP_model.load_state_dict(EXP_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(EXP_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.295, accuracy: 0.511


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(mlp_model,model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 1.533, Accuracy: 0.458
Validation Loss: 1.177, Accuracy: 0.416
Epoch: 2
Train Loss: 1.36, Accuracy: 0.506
Validation Loss: 1.188, Accuracy: 0.46
Epoch: 3
Train Loss: 1.251, Accuracy: 0.541
Validation Loss: 1.207, Accuracy: 0.484
Epoch: 4
Train Loss: 1.172, Accuracy: 0.568
Validation Loss: 1.227, Accuracy: 0.494
Epoch: 5
Train Loss: 1.112, Accuracy: 0.588
Validation Loss: 1.247, Accuracy: 0.499
Epoch: 6
Train Loss: 1.064, Accuracy: 0.605
Validation Loss: 1.265, Accuracy: 0.502
Epoch: 7
Train Loss: 1.025, Accuracy: 0.618
Validation Loss: 1.282, Accuracy: 0.504
Epoch: 8
Train Loss: 0.992, Accuracy: 0.63
Validation Loss: 1.298, Accuracy: 0.505
Epoch: 9
Train Loss: 0.964, Accuracy: 0.639
Validation Loss: 1.312, Accuracy: 0.506
Epoch: 10
Train Loss: 0.939, Accuracy: 0.648
Validation Loss: 1.326, Accuracy: 0.507
7min 37s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.308, accuracy: 0.494


##### Effnet

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(EXP_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.925, Accuracy: 0.644
Validation Loss: 1.389, Accuracy: 0.506
Epoch: 2
Train Loss: 0.745, Accuracy: 0.704
Validation Loss: 1.538, Accuracy: 0.492
Epoch: 3
Train Loss: 0.65, Accuracy: 0.735
Validation Loss: 1.683, Accuracy: 0.485
Epoch: 4
Train Loss: 0.586, Accuracy: 0.757
Validation Loss: 1.792, Accuracy: 0.488
Epoch: 5
Train Loss: 0.539, Accuracy: 0.772
Validation Loss: 1.888, Accuracy: 0.486
Epoch: 6
Train Loss: 0.501, Accuracy: 0.785
Validation Loss: 1.986, Accuracy: 0.487
Epoch: 7
Train Loss: 0.47, Accuracy: 0.796
Validation Loss: 2.051, Accuracy: 0.487
Epoch: 8
Train Loss: 0.443, Accuracy: 0.805
Validation Loss: 2.161, Accuracy: 0.487
Epoch: 9
Train Loss: 0.42, Accuracy: 0.813
Validation Loss: 2.263, Accuracy: 0.477
Epoch: 10
Train Loss: 0.4, Accuracy: 0.82
Validation Loss: 2.335, Accuracy: 0.495
27min 18s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_fusion_{viau}.pth'))
    EXP_model = EXP_fusion().to(device)
    EXP_model.load_state_dict(EXP_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(EXP_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.281, accuracy: 0.506


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 1.596, Accuracy: 0.408
Validation Loss: 1.162, Accuracy: 0.406
Epoch: 2
Train Loss: 1.466, Accuracy: 0.443
Validation Loss: 1.163, Accuracy: 0.441
Epoch: 3
Train Loss: 1.383, Accuracy: 0.468
Validation Loss: 1.18, Accuracy: 0.461
Epoch: 4
Train Loss: 1.322, Accuracy: 0.489
Validation Loss: 1.205, Accuracy: 0.473
Epoch: 5
Train Loss: 1.272, Accuracy: 0.508
Validation Loss: 1.232, Accuracy: 0.478
Epoch: 6
Train Loss: 1.231, Accuracy: 0.523
Validation Loss: 1.257, Accuracy: 0.479
Epoch: 7
Train Loss: 1.196, Accuracy: 0.536
Validation Loss: 1.28, Accuracy: 0.48
Epoch: 8
Train Loss: 1.166, Accuracy: 0.548
Validation Loss: 1.299, Accuracy: 0.48
Epoch: 9
Train Loss: 1.139, Accuracy: 0.558
Validation Loss: 1.316, Accuracy: 0.48
Epoch: 10
Train Loss: 1.116, Accuracy: 0.567
Validation Loss: 1.331, Accuracy: 0.479
6min 53s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: f1_score {f1best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model, strict=False)
    val_loss, f1s, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

best metric: f1_score 0.298, accuracy: 0.461


### Testing

##### EffNet + Wav2vec2

In [ ]:
%%timeit -n 1 -r 1
print('EXP_model')
print(visual_feat + ' & ' + auft)
EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_model/best_EXPR_fusion_{viau}.pth'))
EXP_model = EXP_fusion().to(device)
EXP_model.load_state_dict(EXP_best_model)
test_loss, f1s, acc = evaluate_model(EXP_model, test_loader, auft, weights)
print(f'Test set: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

EXP_model
visualfeat_enet_b2_8_best & audiofeat_wav2vec2
Test set: f1_score 0.33, accuracy: 0.457
34.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_model/best_EXPR_mlp_{viau}.pth'))
mlp_model = MLPModel().to(device)
mlp_model.load_state_dict(mlp_best_model, strict=False)
val_loss, f1s, acc = evaluate_model(mlp_model, test_loader, auft, weights)
print(f'Test set: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

MLP_model
visualfeat_enet_b2_8_best & audiofeat_wav2vec2
Test set: f1_score 0.394, accuracy: 0.488
12.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##### EffNet + Vggish

In [ ]:
%%timeit -n 1 -r 1
print('EXP_model')
print(visual_feat + ' & ' + auft)
EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_model/best_EXPR_fusion_{viau}.pth'))
EXP_model = EXP_fusion().to(device)
EXP_model.load_state_dict(EXP_best_model)
test_loss, f1s, acc = evaluate_model(EXP_model, test_loader, auft, weights)
print(f'Test set: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

EXP_model
visualfeat_enet_b2_8_best & audiofeat_vggish
Test set: f1_score 0.316, accuracy: 0.475
33 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_model/best_EXPR_mlp_{viau}.pth'))
mlp_model = MLPModel().to(device)
mlp_model.load_state_dict(mlp_best_model, strict=False)
val_loss, f1s, acc = evaluate_model(mlp_model, test_loader, auft, weights)
print(f'Test set: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

MLP_model
visualfeat_enet_b2_8_best & audiofeat_vggish
Test set: f1_score 0.379, accuracy: 0.498
11.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##### EffNet

In [ ]:
%%timeit -n 1 -r 1
print('EXP_model')
print(visual_feat + ' & ' + auft)
EXP_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_model/best_EXPR_fusion_{viau}.pth'))
EXP_model = EXP_fusion().to(device)
EXP_model.load_state_dict(EXP_best_model)
test_loss, f1s, acc = evaluate_model(EXP_model, test_loader, auft, weights)
print(f'Test set: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

EXP_model
visualfeat_enet_b2_8_best & nope
Test set: f1_score 0.31, accuracy: 0.447
29.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_model/best_EXPR_mlp_{viau}.pth'))
mlp_model = MLPModel().to(device)
mlp_model.load_state_dict(mlp_best_model, strict=False)
val_loss, f1s, acc = evaluate_model(mlp_model, test_loader, auft, weights)
print(f'Test set: f1_score {round(f1s,3)}, accuracy: {round(acc,3)}')

MLP_model
visualfeat_enet_b2_8_best & nope
Test set: f1_score 0.327, accuracy: 0.431
8.93 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Smoothing

In [ ]:
tsk = task[0]
vis = vis_typ[0]
viau = vis_aud[0]
auft = audio_feat[0]

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, audio_ft = auft, num_classes=8):
        super(MLPModel, self).__init__()
        if audio_ft == 'audiofeat_wav2vec2':
            self.concat_dim = 2176    #1408+768
        elif audio_ft == 'audiofeat_vggish':
            self.concat_dim = 1536    #1408+128
        elif audio_ft == 'nope':
            self.concat_dim = 1408    #visual only
        self.activ = nn.ReLU()
        self.fc1 = nn.Linear(self.concat_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, vis_feat, aud_feat):
        inputs = [vis_feat]
        inputs.append(aud_feat)
        feat = torch.cat(inputs, dim=0)
        feat = self.fc1(feat)
        feat = self.activ(feat)
        out = self.fc2(feat)
        return out, torch.softmax(out, dim=0)

In [ ]:
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_EXPR_model/best_EXPR_mlp_{viau}.pth'))
mlp_model = MLPModel().to(device)
mlp_model.load_state_dict(mlp_best_model)

<All keys matched successfully>

In [ ]:
anno_path = os.path.join(root,f'data/Annotations/{tsk}/{split[1]}')
with open(os.path.join(root, f'data/Annotations/{tsk}/{typ[2]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
with open(os.path.join(root,f'models/ABAW6/{vis}/EXPR/{tsk}_{typ[2]}_{viau}.pkl'), 'rb') as f:
    data = pickle.load(f)

In [ ]:
test_vid = {}
for vname in tqdm(vidnames):
        img, predict, label = [], [], []
        for imgname, val in sorted(data[tsk][vname].items()):
            vis_feat = torch.tensor(val[visual_feat]).to(device)
            if auft == 'nope':
                aud_feat = None
            else:
                aud_feat = torch.tensor(val[auft]).to(device)
            if tsk == task[2]:
                _, _, vpred, apred = mlp_model(vis_feat, aud_feat)
                preds = torch.tensor([vpred, apred])
            else:
                _, pred = mlp_model(vis_feat, aud_feat)
                preds = torch.tensor(pred)
            ind = int(imgname.split('/')[1][:-4])
            img.append(ind)
            predict.append(preds)
            label.append(data[tsk][vname][imgname]['label'])
        test_vid[vname] = (img, predict, label)

100%|██████████| 70/70 [01:22<00:00,  1.18s/it]


In [ ]:
hyperparams=[(isMean,delta) for delta in [0, 5, 15, 30, 60, 100, 200] for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
timing_results = {(isMean, delta): 0 for isMean, delta in hyperparams}
for videoname,(img, predict, label) in test_vid.items():
    for i,ind in enumerate(img):
        total_true.append(label[i].cpu().numpy())
    preds_proba = smooth_prediction(img, predict)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        preds=[]
        start = time.time()
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            if isMean:
                best_ind, proba = slide_window(preds_proba, i, delta, 'mean')
            else:
                best_ind, proba = slide_window(preds_proba, i, delta, 'median')
            preds.append(best_ind)
        for i,ind in enumerate(img):
            if label[i]>=0:
                total_preds[hInd].append(preds[ind-1])
        end = time.time()
        timing_results[(isMean, delta)] += end - start
total_true=np.array(total_true)

In [ ]:
for hInd, (isMean, delta) in enumerate(hyperparams):
    preds = np.array(total_preds[hInd])
    accuracy = round((preds == total_true).mean(), 3)
    f1 = round(f1_score(y_true=total_true, y_pred=preds, average='macro'), 3)
    mean_or_median = 'mean' if isMean else 'median'
    time_taken = round(timing_results[(isMean, delta)],3)
    print(f'{mean_or_median}; delta: {delta}; Acc: {accuracy}; F1: {f1}; Time: {time_taken}s')

mean; delta: 0; Acc: 0.488; F1: 0.394; Time: 7.318s
mean; delta: 5; Acc: 0.512; F1: 0.418; Time: 7.51s
median; delta: 5; Acc: 0.509; F1: 0.417; Time: 16.631s
mean; delta: 15; Acc: 0.524; F1: 0.429; Time: 7.611s
median; delta: 15; Acc: 0.523; F1: 0.43; Time: 17.372s
mean; delta: 30; Acc: 0.533; F1: 0.44; Time: 8.184s
median; delta: 30; Acc: 0.533; F1: 0.442; Time: 18.457s
mean; delta: 60; Acc: 0.537; F1: 0.443; Time: 7.855s
median; delta: 60; Acc: 0.538; F1: 0.449; Time: 20.305s
mean; delta: 100; Acc: 0.539; F1: 0.448; Time: 8.482s
median; delta: 100; Acc: 0.541; F1: 0.454; Time: 22.453s
mean; delta: 200; Acc: 0.538; F1: 0.437; Time: 10.033s
median; delta: 200; Acc: 0.544; F1: 0.457; Time: 27.948s


In [ ]:
for hInd, (isMean, delta) in enumerate(hyperparams):
    preds = np.array(total_preds[hInd])
    accuracy = round((preds == total_true).mean(), 3)
    f1 = round(f1_score(y_true=total_true, y_pred=preds, average='macro'), 3)
    mean_or_median = 'mean' if isMean else 'median'
    time_taken = round(timing_results[(isMean, delta)],3)
    print(f'{mean_or_median}; delta: {delta}; Acc: {accuracy}; F1: {f1}; Time: {time_taken}s')

mean; delta: 0; Acc: 0.488; F1: 0.394; Time: 9.554s
mean; delta: 15; Acc: 0.524; F1: 0.429; Time: 9.97s
median; delta: 15; Acc: 0.523; F1: 0.43; Time: 20.216s
mean; delta: 30; Acc: 0.533; F1: 0.44; Time: 9.814s
median; delta: 30; Acc: 0.533; F1: 0.442; Time: 20.944s
mean; delta: 60; Acc: 0.537; F1: 0.443; Time: 9.286s
median; delta: 60; Acc: 0.538; F1: 0.449; Time: 22.854s
mean; delta: 100; Acc: 0.539; F1: 0.448; Time: 9.671s
median; delta: 100; Acc: 0.541; F1: 0.454; Time: 25.641s
mean; delta: 200; Acc: 0.538; F1: 0.437; Time: 11.106s
median; delta: 200; Acc: 0.544; F1: 0.457; Time: 31.662s


### Adaptive Frame Rate

In [ ]:
delta = 200

In [ ]:
anno_path = os.path.join(root,f'data/Annotations/{tsk}/{split[0]}')
with open(os.path.join(root, f'data/Annotations/{tsk}/{typ[0]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
with open(os.path.join(root,f'models/ABAW6/{vis}/EXPR/{tsk}_{typ[0]}_{viau}.pkl'), 'rb') as f:
    data = pickle.load(f)
train_vid = {}
for vname in tqdm(vidnames):
        img, predict, label = [], [], []
        for imgname, val in sorted(data[tsk][vname].items()):
            vis_feat = torch.tensor(val[visual_feat]).to(device)
            if auft == 'nope':
                aud_feat = None
            else:
                aud_feat = torch.tensor(val[auft]).to(device)
            if tsk == task[2]:
                _, _, vpred, apred = mlp_model(vis_feat, aud_feat)
                preds = torch.tensor([vpred, apred])
            else:
                _, pred = mlp_model(vis_feat, aud_feat)
                preds = torch.tensor(pred)
            ind = int(imgname.split('/')[1][:-4])
            img.append(ind)
            predict.append(preds)
            label.append(data[tsk][vname][imgname]['label'])
        train_vid[vname] = (img, predict, label)

100%|██████████| 199/199 [02:23<00:00,  1.39it/s]


In [ ]:
stride2scores={}
for stride in [200, 100, 50, 25, 10]:
    total_true, predictions, max_decision_values = [],[],[]
    for vidname, (img, predict, label) in train_vid.items():
        index = []
        for i,ind in enumerate(img):
            total_true.append(label[i].cpu().numpy())
            index.append(ind-1)
        preds_proba = smooth_prediction(img, predict)
        for i in range(len(index)):
            best_ind, proba = slide_window(preds_proba, index[i], delta, stride)
            predictions.append(best_ind)
            max_decision_values.append(proba[best_ind])
    stride2scores[stride] = (np.array(total_true),np.array(predictions),np.array(max_decision_values))

In [ ]:
def get_threshold(stride,fpr_corrected):
    (total_true,predictions,max_decision_values) = stride2scores[stride]
    mistakes = max_decision_values[predictions != total_true]
    best_threshold = -1
    for i, threshold in enumerate(sorted(max_decision_values[predictions == total_true])[::-1]):
        tpr = i/len(predictions)
        fpr = (mistakes > threshold).sum()/len(predictions)
        if fpr > fpr_corrected:
            if best_threshold == -1:
                best_threshold = threshold
            print(stride, 'best_threshold', best_threshold, i)
            break
        best_threshold = threshold
    return best_threshold

In [ ]:
stride2threshold = {}
for stride in stride2scores:
    fpr_corrected=0.05
    stride2threshold[stride] = get_threshold(stride,fpr_corrected)
stride2threshold[1] = 0
print(stride2threshold)

200 best_threshold 0.5776766 230724
100 best_threshold 0.5594592 241835
50 best_threshold 0.54468054 251611
25 best_threshold 0.538801 255266
10 best_threshold 0.53340304 259316
{200: 0.5776766, 100: 0.5594592, 50: 0.54468054, 25: 0.538801, 10: 0.53340304, 1: 0}


In [ ]:
all_strides=[
    [200, 100, 50, 10, 1],
    [50, 25, 1],
    [50, 10, 1],
    [200,50,1],
    [100,50,1],
    [200,1],
    [100,1],
    [50,1]
]
for s in stride2threshold.keys():
    all_strides.append([s])

for strides in all_strides:
    print(strides)
    last_stride=strides[-1]

    total_true=[]
    total_preds=[]
    total_frames_processed,total_frames=0,0
    time_each = []
    start = time.time()
    for videoname, (img, predict, label) in test_vid.items():
        emotional_img=[]
        start1 = time.time()
        for i,ind in enumerate(img):
            total_true.append(label[i].cpu().numpy())
            emotional_img.append(ind-1)
        cur_ind=0
        preds_proba=[]
        for i in range(img[-1]):
            if img[cur_ind]-1==i:
                preds_proba.append(predict[cur_ind])
                cur_ind+=1
            else:
                if cur_ind==0:
                    preds_proba.append(predict[cur_ind])
                else:
                    w=(i-img[cur_ind-1]+1)/(img[cur_ind]-img[cur_ind-1])
                    pred=w*predict[cur_ind-1]+(1-w)*predict[cur_ind]
                    preds_proba.append(pred)

        preds_proba=np.array([p.cpu().numpy() for p in preds_proba])

        preds=-np.ones(len(emotional_img))
        end1 = time.time()
        time_each.append(end1 - start1)
        for stride in strides:
            threshold=stride2threshold[stride]
            for i in range(len(emotional_img)):
                if preds[i]<0:
                    i1=max(emotional_img[i]-delta,0)
                    cur_preds=preds_proba[i1:emotional_img[i]+delta+1:stride]
                    proba=np.median(cur_preds,axis=0)
                    best_ind=np.argmax(proba)
                    if proba[best_ind]>=threshold or stride==last_stride:
                        total_frames_processed+=len(cur_preds)
                        total_frames+=len(preds_proba[i1:emotional_img[i]+delta+1])
                        preds[i]=best_ind
        for p in preds:
            total_preds.append(p)
    end = time.time()
    elapsed_time = end - start - sum(time_each)
    total_true=np.array(total_true)
    preds=np.array(total_preds)
    print('Acc:',round((preds==total_true).mean(),3), 'F1:',round(f1_score(y_true=total_true,y_pred=preds, average="macro"),3))
    print(total_frames_processed,total_frames,round(total_frames_processed/total_frames,3))
    print(f"Time: {elapsed_time:.2f} seconds")

[200, 100, 50, 10, 1]
Acc: 0.543 F1: 0.454
66718832 109811197 0.608
Time: 50.22 seconds
[50, 25, 1]
Acc: 0.544 F1: 0.457
75888291 109811197 0.691
Time: 35.62 seconds
[50, 10, 1]
Acc: 0.544 F1: 0.457
74809429 109811197 0.681
Time: 35.97 seconds
[200, 50, 1]
Acc: 0.543 F1: 0.455
72017266 109811197 0.656
Time: 34.61 seconds
[100, 50, 1]
Acc: 0.544 F1: 0.456
75050361 109811197 0.683
Time: 35.59 seconds
[200, 1]
Acc: 0.543 F1: 0.455
81565887 109811197 0.743
Time: 29.01 seconds
[100, 1]
Acc: 0.544 F1: 0.456
80951438 109811197 0.737
Time: 28.60 seconds
[50, 1]
Acc: 0.544 F1: 0.457
79911794 109811197 0.728
Time: 28.14 seconds
[200]
Acc: 0.498 F1: 0.411
816872 109811197 0.007
Time: 13.12 seconds
[100]
Acc: 0.514 F1: 0.429
1364594 109811197 0.012
Time: 13.06 seconds
[50]
Acc: 0.527 F1: 0.443
2459930 109811197 0.022
Time: 13.21 seconds
[25]
Acc: 0.536 F1: 0.45
4650733 109811197 0.042
Time: 13.65 seconds
[10]
Acc: 0.541 F1: 0.455
11223222 109811197 0.102
Time: 13.91 seconds
[1]
Acc: 0.544 F1: 0.45

## AU Detection Challenge

### Loading data

In [ ]:
# Cropped_aligned images
vis = vis_typ[0]
vis1 = vis_typ[2]

#### Effnet + wav2vec2

In [ ]:
auft = audio_feat[0]
viau = vis_aud[0]

#### Effnet + vggish

In [ ]:
auft = audio_feat[1]
viau = vis_aud[1]

#### Effnet

In [ ]:
auft = audio_feat[2]
viau = vis_aud[2]

#### Train

In [ ]:
if viau == vis_aud[0]: #Visual+wav2vec2
    with open(os.path.join(root,f'models/ABAW6/{vis}/AU/{task[1]}_{typ[0]}_visual.pkl'), 'rb') as f:
        data1 = pickle.load(f)
    with open(os.path.join(root, f'data/Annotations/{task[1]}/{typ[0]}.txt'), 'r') as f:
        vidnames = f.read().splitlines()
    task1 = task[1]
    feature_a = 'audiofeat_wav2vec2'
    feat_root = os.path.join(root + '/models/ABAW6', feature_a)
    filenames = os.listdir(feat_root)[:]
    for vname in tqdm(vidnames):
            feature = np.load(os.path.join(feat_root, f'{vname}.npy'), allow_pickle=True).tolist()
            for imgname, val in feature.items():
                if imgname in data1[task1][vname]:
                    data1[task1][vname][imgname].update({f'{feature_a}': val})
            for img, value in list(data1[task1][vname].items()):
                if len(value) < 3:
                    data1[task1][vname].pop(img)
else:
    with open(os.path.join(root,f'models/ABAW6/{vis}/AU/{task[1]}_{typ[0]}_{viau}.pkl'), 'rb') as f:
        data1 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[1]
with open(os.path.join(root, f'data/Annotations/{task[1]}/{typ[0]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data1[task1][vname])
    for img in data1[task1][vname].keys():
        iname.append(img)

100%|██████████| 236/236 [00:00<00:00, 1347.13it/s]


In [ ]:
dataset = ABAW_dataset1(data1, iname, dims, task1)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0, drop_last=True)

#### Val

In [ ]:
with open(os.path.join(root,f'models/ABAW6/{vis}/AU/{task[1]}_{typ[1]}_{viau}.pkl'), 'rb') as f:
    data2 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[1]
with open(os.path.join(root, f'data/Annotations/{task[1]}/{typ[1]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data2[task1][vname])
    for img in data2[task1][vname].keys():
        iname.append(img)

100%|██████████| 59/59 [00:00<00:00, 1230.53it/s]


In [ ]:
dataset = ABAW_dataset1(data2, iname, dims, task1)
val_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, drop_last=False)

#### Test

##### EffNet_B2

In [ ]:
with open(os.path.join(root,f'models/ABAW6/{vis}/AU/{task[1]}_{typ[2]}_{viau}.pkl'), 'rb') as f:
    data3 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[1]
with open(os.path.join(root, f'data/Annotations/{task[1]}/{typ[2]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data3[task1][vname])
    for img in data3[task1][vname].keys():
        iname.append(img)

100%|██████████| 105/105 [00:00<00:00, 1203.40it/s]


In [ ]:
dataset = ABAW_dataset1(data3, iname, dims, task1)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, drop_last=False)

##### EffNet_B0

In [ ]:
with open(os.path.join(root,f'models/ABAW6/{vis1}/AU/{task[1]}_{typ[2]}_{viau}.pkl'), 'rb') as f:
    data3 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[1]
with open(os.path.join(root, f'data/Annotations/{task[1]}/{typ[2]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data3[task1][vname])
    for img in data3[task1][vname].keys():
        iname.append(img)

In [ ]:
dataset = ABAW_dataset1(data3, iname, dims, task1)
test_loader_b0 = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, drop_last=False)

### Modeling

In [ ]:
class AU_fusion(nn.Module):
    def __init__(self, batchsize = batch_size, audio_ft = auft, hidden_size = [512, 128, batch_size]):
        super(AU_fusion, self).__init__()
        self.batchsize = batchsize
        if audio_ft == 'audiofeat_wav2vec2':
            self.concat_dim = 2176    #1408+768
        elif audio_ft == 'audiofeat_vggish':
            self.concat_dim = 1536    #1408+128
        elif audio_ft == 'nope':
            self.concat_dim = 1408    #visual only
        self.hidden_size = hidden_size
        self.feat_fc = nn.Conv1d(self.concat_dim, hidden_size[0], 1, padding=0)
        self.activ = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(p=0.3)
        self.conv1 = nn.Conv1d(hidden_size[0], hidden_size[1], 1, padding=0)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size[1], nhead=4, dim_feedforward=hidden_size[1], dropout=0.3)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)
        self.head = nn.Sequential(
                nn.Linear(hidden_size[1], hidden_size[2]),
                nn.BatchNorm1d(hidden_size[2]),
                nn.Linear(hidden_size[2], 12))

    def forward(self, vis_feat, aud_feat):
        if aud_feat == None:
            feat = vis_feat
        else:
            inputs = [vis_feat]
            inputs.append(aud_feat)
            feat = torch.cat(inputs,dim=1)
        feat = torch.transpose(feat,0,1)
        feat = self.feat_fc(feat)
        feat = self.activ(feat)
        out = self.conv1(feat)
        out = torch.transpose(out,0,1)
        out = self.transformer_encoder(out)
        out = self.head(out)

        return out, torch.sigmoid(out)

In [ ]:
AU_model = AU_fusion().to(device)
AU_model

AU_fusion(
  (feat_fc): Conv1d(1408, 512, kernel_size=(1,), stride=(1,))
  (activ): LeakyReLU(negative_slope=0.1)
  (dropout): Dropout(p=0.3, inplace=False)
  (conv1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=128, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
    )
  )
  (head): Sequential(
    (0): Linear(in_features=128, out_features

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, audio_ft = auft, num_classes=12):
        super(MLPModel, self).__init__()
        if audio_ft == 'audiofeat_wav2vec2':
            self.concat_dim = 2176    #1408+768
        elif audio_ft == 'audiofeat_vggish':
            self.concat_dim = 1536    #1408+128
        elif audio_ft == 'nope':
            self.concat_dim = 1408    #visual only
        self.activ = nn.ReLU()
        self.fc1 = nn.Linear(self.concat_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, vis_feat, aud_feat):
        if aud_feat == None:
            feat = vis_feat
        else:
            inputs = [vis_feat]
            inputs.append(aud_feat)
            feat = torch.cat(inputs, dim=1)
        feat = self.fc1(feat)
        feat = self.activ(feat)
        out = self.fc2(feat)
        return out, torch.sigmoid(out)

In [ ]:
class MLPModel_b0(nn.Module):
    def __init__(self, audio_ft = auft, num_classes=12):
        super(MLPModel_b0, self).__init__()
        if audio_ft == 'audiofeat_wav2vec2':
            self.concat_dim = 2048 #1280+768
        elif audio_ft == 'audiofeat_vggish':
            self.concat_dim = 1408 #1280+128
        elif audio_ft == 'nope':
            self.concat_dim = 1280 #1280    #visual only
        self.activ = nn.ReLU()
        self.fc1 = nn.Linear(self.concat_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, vis_feat, aud_feat):
        if aud_feat == None:
            feat = vis_feat
        else:
            inputs = [vis_feat]
            inputs.append(aud_feat)
            feat = torch.cat(inputs, dim=1)
        feat = self.fc1(feat)
        feat = self.activ(feat)
        out = self.fc2(feat)
        return out, torch.sigmoid(out)

In [ ]:
mlp_model = MLPModel().to(device)
mlp_model

MLPModel(
  (activ): ReLU()
  (fc1): Linear(in_features=1536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=12, bias=True)
)

#### Utils

In [ ]:
weights1 = torch.tensor([0.54733899, 0.44180561, 0.56990565, 0.61997328, 0.73956417,0.74692377, 0.72684634, 0.33222808, 0.17383676, 0.20608964, 0.83688068, 0.33890931]).to(device)

In [ ]:
y_train = []
iterator = iter(train_loader)
i = 0
while True:
    try:
        AU = next(iterator)
        y_train.extend(AU['label'].numpy())
    except:
        break

In [ ]:
y_train = np.array(y_train)
weights = np.empty([12, 2])

for i in range(12):
    neg, pos = np.bincount(y_train[:, i])
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0) if neg > 0 else 0
    weight_for_1 = (1 / pos) * (total / 2.0) if pos > 0 else 0

    weights[i][0] = weight_for_0
    weights[i][1] = weight_for_1
print(weights)

[[ 0.57226378  3.95954782]
 [ 0.52683025  9.81784119]
 [ 0.59477198  3.13791059]
 [ 0.67850865  1.90049238]
 [ 0.82921345  1.25938575]
 [ 0.75994361  1.46174704]
 [ 0.65729978  2.089322  ]
 [ 0.51501945 17.1450862 ]
 [ 0.51587402 16.24900868]
 [ 0.51460027 17.62297096]
 [ 1.33900704  0.79797128]
 [ 0.54113295  6.57785288]]


In [ ]:
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, AU_model.parameters()), lr=0.00001, betas=(0.9, 0.999), weight_decay=0.00005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=1e-5)

In [ ]:
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, mlp_model.parameters()), lr=0.0001, betas=(0.9, 0.999), weight_decay=0.00005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=1e-5)

### Training

In [ ]:
def train(model, mod_type, train_loader, val_loader, epoch, batch_size, optim, au_feat, weight, vi_au):
    model.train(True)
    model.eval()
    best_loss = float('inf')
    f1s_best, accbest = 0, 0
    loss_value = []
    loss_train = []
    loss_val = []
    all_preds = []
    all_targets = []

    for e in range(epoch):
        print(f'Epoch: {e+1}')
        torch.manual_seed(2809)
        iterator = iter(train_loader)
        for AU in iterator:
                if au_feat == 'nope':
                    vis_feat, y = AU[visual_feat_1], AU['label']
                    vis_feat, y = vis_feat.to(device), y.to(device)
                    aud_feat = None
                else:
                    vis_feat, aud_feat, y = AU[visual_feat_1], AU[au_feat], AU['label']
                    vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
                model.zero_grad()
                pred, au_pred = model(vis_feat, aud_feat)
                loss = compute_AU_loss(pred, y, weight)
                loss.backward()
                optim.step()
                loss_value.append(loss.item())
                all_preds.extend(au_pred.cpu().tolist())
                all_targets.extend(y.cpu().tolist())

        avg_loss = round(np.mean(loss_value),3)
        loss_train.append(avg_loss)
        f1_scores, f1_thresh, accuracy, threshold = compute_AU_F1(all_preds, all_targets)
        print(f'Train Loss: {avg_loss}, Accuracy of 12 AU classes: {accuracy}')

        val_loss, f1s, f1t, acc, f1_threshold = evaluate_model(model, val_loader, visual_feature, au_feat, weight)
        loss_val.append(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), os.path.join(root,f'models/ABAW6/{vis}/best_AU_{mod_type}_{vi_au}.pth'))
            f1best = f1s
            accbest = acc

        print(f'Validation Loss: {val_loss}, Accuracy of 12 AU classes: {acc}')
        scheduler.step(val_loss)
    return loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest

In [ ]:
def evaluate_model(model, data_loader, vi_feat, au_feat, weight):
    model.eval()
    total_loss = []
    all_preds = []
    all_targets = []
    with torch.no_grad():
        iterator = iter(data_loader)
        for AU in iterator:
            if au_feat == 'nope':
                vis_feat, y = AU[vi_feat], AU['label']
                vis_feat, y = vis_feat.to(device), y.to(device)
                aud_feat = None
            else:
                vis_feat, aud_feat, y = AU[vi_feat], AU[au_feat], AU['label']
                vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
            pred, au_pred = model(vis_feat, aud_feat)
            loss = compute_AU_loss(pred, y, weight)
            total_loss.append(loss.item())
            all_preds.extend(au_pred.cpu().tolist())
            all_targets.extend(y.cpu().tolist())

    f1_scores, f1_thresh, acc, threshold = compute_AU_F1(all_preds, all_targets)
    return round(np.mean(total_loss),3), round(f1_scores,3), round(f1_thresh,3), acc, threshold

#### Cropped_aligned images

##### EffNet + Wav2vec2

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(AU_model, model_type[0], train_loader, val_loader, 8, 32, optimizer, auft, weights1, viau)

Epoch: 1
Train Loss: 0.141, Accuracy of 12 AU classes: [0.905, 0.949, 0.929, 0.882, 0.807, 0.828, 0.896, 0.936, 0.878, 0.916, 0.82, 0.895]
Validation Loss: 0.177, Accuracy of 12 AU classes: [0.914, 0.947, 0.866, 0.797, 0.744, 0.79, 0.861, 0.883, 0.919, 0.976, 0.777, 0.913]
Epoch: 2
Train Loss: 0.128, Accuracy of 12 AU classes: [0.914, 0.953, 0.938, 0.897, 0.822, 0.841, 0.906, 0.968, 0.908, 0.973, 0.835, 0.927]
Validation Loss: 0.184, Accuracy of 12 AU classes: [0.914, 0.947, 0.867, 0.781, 0.745, 0.77, 0.859, 0.911, 0.925, 0.976, 0.773, 0.914]
Epoch: 3
Train Loss: 0.12, Accuracy of 12 AU classes: [0.92, 0.956, 0.944, 0.905, 0.831, 0.848, 0.912, 0.97, 0.917, 0.977, 0.844, 0.93]
Validation Loss: 0.191, Accuracy of 12 AU classes: [0.912, 0.945, 0.866, 0.78, 0.725, 0.787, 0.856, 0.921, 0.929, 0.976, 0.773, 0.916]
Epoch: 4
Train Loss: 0.115, Accuracy of 12 AU classes: [0.93, 0.958, 0.947, 0.911, 0.838, 0.854, 0.916, 0.972, 0.921, 0.979, 0.851, 0.932]
Validation Loss: 0.196, Accuracy of 12 AU

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_fusion_{viau}_loss.pth'))
    AU_model = AU_fusion().to(device)
    AU_model.load_state_dict(AU_best_model)
    val_loss, f1s, f1t, acc, threshold = evaluate_model(AU_model, val_loader, auft, weights1)
    print(f'best metric: f1_score {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

best metric: f1_score 0.487, accuracy: [0.914, 0.947, 0.866, 0.797, 0.744, 0.79, 0.861, 0.883, 0.919, 0.976, 0.777, 0.913], f1_threshold: [0.30000000000000004, 0.2, 0.2, 0.4, 0.5, 0.4, 0.4, 0.1, 0.1, 0.2, 0.4, 0.2]


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 8, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.488, Accuracy of 12 AU classes: [0.884, 0.943, 0.868, 0.793, 0.754, 0.771, 0.865, 0.947, 0.961, 0.946, 0.759, 0.899]
Validation Loss: 0.472, Accuracy of 12 AU classes: [0.927, 0.95, 0.86, 0.797, 0.73, 0.775, 0.851, 0.89, 0.841, 0.973, 0.777, 0.908]
Epoch: 2
Train Loss: 0.456, Accuracy of 12 AU classes: [0.889, 0.942, 0.892, 0.832, 0.767, 0.784, 0.87, 0.951, 0.947, 0.967, 0.776, 0.896]
Validation Loss: 0.468, Accuracy of 12 AU classes: [0.925, 0.944, 0.862, 0.8, 0.735, 0.78, 0.856, 0.906, 0.914, 0.969, 0.785, 0.903]
Epoch: 3
Train Loss: 0.437, Accuracy of 12 AU classes: [0.892, 0.941, 0.898, 0.839, 0.774, 0.79, 0.872, 0.952, 0.939, 0.967, 0.785, 0.895]
Validation Loss: 0.469, Accuracy of 12 AU classes: [0.924, 0.941, 0.864, 0.8, 0.737, 0.783, 0.858, 0.91, 0.911, 0.966, 0.787, 0.901]
Epoch: 4
Train Loss: 0.424, Accuracy of 12 AU classes: [0.894, 0.941, 0.902, 0.845, 0.78, 0.795, 0.874, 0.952, 0.934, 0.968, 0.791, 0.895]
Validation Loss: 0.47, Accuracy of 12 AU clas

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_mlp_{viau}_acc.pth'))
    mlp_model = MLPModel(num_classes = 12).to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, f1s, f1t, acc, threshold = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

best metric: f1_score 0.504, accuracy: [0.922, 0.953, 0.871, 0.799, 0.742, 0.786, 0.86, 0.917, 0.91, 0.98, 0.775, 0.923], f1_threshold: [0.8, 0.9, 0.6, 0.6, 0.5, 0.5, 0.6, 0.7000000000000001, 0.7000000000000001, 0.9, 0.2, 0.8]


##### EffNet + Vggish

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(AU_model, model_type[0], train_loader, val_loader, 8, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.373, Accuracy of 12 AU classes: [0.902, 0.942, 0.919, 0.866, 0.797, 0.808, 0.882, 0.97, 0.949, 0.978, 0.802, 0.896]
Validation Loss: 0.51, Accuracy of 12 AU classes: [0.918, 0.952, 0.884, 0.794, 0.744, 0.78, 0.87, 0.946, 0.913, 0.968, 0.785, 0.915]
Epoch: 2
Train Loss: 0.327, Accuracy of 12 AU classes: [0.921, 0.945, 0.929, 0.88, 0.811, 0.82, 0.899, 0.97, 0.965, 0.981, 0.818, 0.924]
Validation Loss: 0.581, Accuracy of 12 AU classes: [0.912, 0.95, 0.878, 0.795, 0.727, 0.778, 0.865, 0.936, 0.915, 0.971, 0.768, 0.913]
Epoch: 3
Train Loss: 0.3, Accuracy of 12 AU classes: [0.926, 0.96, 0.934, 0.889, 0.819, 0.827, 0.905, 0.97, 0.965, 0.983, 0.827, 0.928]
Validation Loss: 0.651, Accuracy of 12 AU classes: [0.913, 0.95, 0.877, 0.785, 0.734, 0.78, 0.861, 0.945, 0.919, 0.963, 0.769, 0.899]
Epoch: 4
Train Loss: 0.281, Accuracy of 12 AU classes: [0.93, 0.962, 0.942, 0.895, 0.825, 0.833, 0.909, 0.971, 0.965, 0.984, 0.834, 0.931]
Validation Loss: 0.719, Accuracy of 12 AU class

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_model/best_AU_fusion_{viau}.pth'))
    AU_model = AU_fusion().to(device)
    AU_model.load_state_dict(AU_best_model)
    val_loss, f1s, f1t, acc, threshold = evaluate_model(AU_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

best metric: f1_score 0.494, accuracy: [0.919, 0.949, 0.876, 0.779, 0.76, 0.77, 0.861, 0.956, 0.882, 0.972, 0.777, 0.918], f1_threshold: [0.30000000000000004, 0.2, 0.2, 0.30000000000000004, 0.5, 0.30000000000000004, 0.4, 0.2, 0.1, 0.2, 0.4, 0.2]


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 8, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.501, Accuracy of 12 AU classes: [0.854, 0.943, 0.863, 0.788, 0.75, 0.768, 0.864, 0.892, 0.91, 0.95, 0.751, 0.896]
Validation Loss: 0.469, Accuracy of 12 AU classes: [0.908, 0.952, 0.865, 0.795, 0.739, 0.781, 0.864, 0.884, 0.841, 0.977, 0.778, 0.91]
Epoch: 2
Train Loss: 0.47, Accuracy of 12 AU classes: [0.885, 0.942, 0.87, 0.801, 0.764, 0.78, 0.868, 0.95, 0.95, 0.968, 0.769, 0.893]
Validation Loss: 0.461, Accuracy of 12 AU classes: [0.927, 0.95, 0.865, 0.799, 0.743, 0.786, 0.855, 0.902, 0.845, 0.973, 0.785, 0.909]
Epoch: 3
Train Loss: 0.452, Accuracy of 12 AU classes: [0.888, 0.941, 0.891, 0.81, 0.771, 0.786, 0.87, 0.95, 0.942, 0.968, 0.778, 0.893]
Validation Loss: 0.459, Accuracy of 12 AU classes: [0.927, 0.949, 0.884, 0.8, 0.743, 0.789, 0.868, 0.906, 0.918, 0.97, 0.787, 0.909]
Epoch: 4
Train Loss: 0.44, Accuracy of 12 AU classes: [0.891, 0.941, 0.894, 0.838, 0.775, 0.791, 0.872, 0.95, 0.937, 0.968, 0.784, 0.893]
Validation Loss: 0.459, Accuracy of 12 AU classes:

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_mlp_{viau}.pth'))
    mlp_model = MLPModel(num_classes = 12).to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, f1s, f1t, acc, threshold = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

best metric: f1_score 0.515, accuracy: [0.924, 0.945, 0.872, 0.802, 0.72, 0.791, 0.869, 0.948, 0.916, 0.98, 0.789, 0.929], f1_threshold: [0.8, 0.8, 0.6, 0.6, 0.4, 0.5, 0.7000000000000001, 0.8, 0.7000000000000001, 0.9, 0.30000000000000004, 0.8]


##### EffNet

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(AU_model, model_type[0], train_loader, val_loader, 8, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.364, Accuracy of 12 AU classes: [0.901, 0.939, 0.92, 0.867, 0.8, 0.814, 0.893, 0.971, 0.949, 0.979, 0.807, 0.921]
Validation Loss: 0.524, Accuracy of 12 AU classes: [0.917, 0.946, 0.867, 0.785, 0.718, 0.782, 0.851, 0.943, 0.932, 0.968, 0.769, 0.916]
Epoch: 2
Train Loss: 0.323, Accuracy of 12 AU classes: [0.92, 0.956, 0.929, 0.88, 0.814, 0.826, 0.9, 0.971, 0.965, 0.981, 0.823, 0.924]
Validation Loss: 0.576, Accuracy of 12 AU classes: [0.915, 0.948, 0.875, 0.771, 0.736, 0.788, 0.855, 0.953, 0.919, 0.973, 0.77, 0.917]
Epoch: 3
Train Loss: 0.299, Accuracy of 12 AU classes: [0.925, 0.958, 0.934, 0.887, 0.822, 0.833, 0.904, 0.971, 0.965, 0.982, 0.832, 0.927]
Validation Loss: 0.626, Accuracy of 12 AU classes: [0.914, 0.948, 0.871, 0.772, 0.741, 0.789, 0.845, 0.943, 0.921, 0.976, 0.758, 0.903]
Epoch: 4
Train Loss: 0.283, Accuracy of 12 AU classes: [0.928, 0.96, 0.942, 0.893, 0.828, 0.838, 0.908, 0.971, 0.965, 0.983, 0.838, 0.93]
Validation Loss: 0.674, Accuracy of 12 AU 

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_fusion_{viau}_loss.pth'))
    AU_model = AU_fusion().to(device)
    AU_model.load_state_dict(AU_best_model)
    val_loss, f1s, f1t, acc, threshold = evaluate_model(AU_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

best metric: f1_score 0.495, accuracy: [0.917, 0.946, 0.867, 0.785, 0.718, 0.782, 0.851, 0.943, 0.932, 0.968, 0.769, 0.916], f1_threshold: [0.8, 0.8, 0.5, 0.6, 0.4, 0.5, 0.7000000000000001, 0.7000000000000001, 0.7000000000000001, 0.8, 0.30000000000000004, 0.7000000000000001]


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 8, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.507, Accuracy of 12 AU classes: [0.878, 0.941, 0.86, 0.782, 0.701, 0.767, 0.863, 0.891, 0.908, 0.947, 0.747, 0.894]
Validation Loss: 0.476, Accuracy of 12 AU classes: [0.905, 0.95, 0.878, 0.795, 0.726, 0.779, 0.853, 0.885, 0.846, 0.975, 0.774, 0.904]
Epoch: 2
Train Loss: 0.479, Accuracy of 12 AU classes: [0.883, 0.939, 0.865, 0.794, 0.761, 0.778, 0.866, 0.951, 0.95, 0.966, 0.765, 0.891]
Validation Loss: 0.47, Accuracy of 12 AU classes: [0.904, 0.947, 0.879, 0.798, 0.733, 0.784, 0.854, 0.899, 0.921, 0.973, 0.781, 0.928]
Epoch: 3
Train Loss: 0.462, Accuracy of 12 AU classes: [0.887, 0.939, 0.87, 0.803, 0.767, 0.785, 0.868, 0.951, 0.942, 0.966, 0.775, 0.89]
Validation Loss: 0.468, Accuracy of 12 AU classes: [0.923, 0.944, 0.882, 0.797, 0.737, 0.787, 0.854, 0.902, 0.918, 0.971, 0.783, 0.928]
Epoch: 4
Train Loss: 0.45, Accuracy of 12 AU classes: [0.889, 0.939, 0.89, 0.809, 0.771, 0.789, 0.869, 0.951, 0.937, 0.967, 0.781, 0.89]
Validation Loss: 0.469, Accuracy of 12 AU

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_model/best_AU_mlp_{viau}.pth'))
    mlp_model = MLPModel(num_classes = 12).to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, f1s, f1t, acc, threshold = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

best metric: f1_score 0.51, accuracy: [0.92, 0.957, 0.885, 0.794, 0.741, 0.788, 0.865, 0.95, 0.917, 0.981, 0.784, 0.926], f1_threshold: [0.8, 0.9, 0.7000000000000001, 0.6, 0.5, 0.5, 0.7000000000000001, 0.8, 0.7000000000000001, 0.9, 0.30000000000000004, 0.8]


#### Cropped images

##### EffNet + Wav2vec2

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(AU_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.248, Accuracy of 12 AU classes: [0.464, 0.616, 0.696, 0.494, 0.59, 0.726, 0.822, 0.821, 0.398, 0.027, 0.698, 0.474]
Validation Loss: 0.201, Accuracy of 12 AU classes: [0.84, 0.875, 0.523, 0.893, 0.783, 0.88, 0.895, 0.906, 0.176, 0.55, 0.755, 0.465]
Epoch: 2
Train Loss: 0.224, Accuracy of 12 AU classes: [0.863, 0.773, 0.788, 0.641, 0.668, 0.754, 0.838, 0.881, 0.228, 0.089, 0.721, 0.633]
Validation Loss: 0.177, Accuracy of 12 AU classes: [0.913, 0.916, 0.754, 0.884, 0.826, 0.877, 0.898, 0.955, 0.176, 0.333, 0.743, 0.734]
Epoch: 3
Train Loss: 0.21, Accuracy of 12 AU classes: [0.872, 0.828, 0.82, 0.698, 0.701, 0.767, 0.847, 0.769, 0.135, 0.3, 0.735, 0.705]
Validation Loss: 0.168, Accuracy of 12 AU classes: [0.91, 0.919, 0.768, 0.888, 0.833, 0.881, 0.892, 0.978, 0.219, 0.754, 0.755, 0.867]
Epoch: 4
Train Loss: 0.2, Accuracy of 12 AU classes: [0.877, 0.944, 0.838, 0.808, 0.72, 0.776, 0.853, 0.812, 0.274, 0.437, 0.746, 0.889]
Validation Loss: 0.164, Accuracy of 12 AU cl

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_fusion_{viau}.pth'))
    AU_model = AU_fusion().to(device)
    AU_model.load_state_dict(AU_best_model)
    val_loss, f1s, f1t, acc = evaluate_model(AU_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, f1_threshold: {f1t}, accuracy: {acc}')

best metric: f1_score 0.552, f1_threshold: 0.258, accuracy: [0.92, 0.923, 0.814, 0.894, 0.846, 0.884, 0.88, 0.946, 0.505, 0.846, 0.759, 0.896]


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.287, Accuracy of 12 AU classes: [0.126, 0.109, 0.306, 0.452, 0.535, 0.368, 0.505, 0.146, 0.073, 0.277, 0.63, 0.078]
Validation Loss: 0.24, Accuracy of 12 AU classes: [0.12, 0.577, 0.138, 0.878, 0.683, 0.843, 0.855, 0.933, 0.513, 0.089, 0.645, 0.089]
Epoch: 2
Train Loss: 0.256, Accuracy of 12 AU classes: [0.182, 0.47, 0.557, 0.607, 0.62, 0.491, 0.664, 0.294, 0.466, 0.362, 0.637, 0.24]
Validation Loss: 0.215, Accuracy of 12 AU classes: [0.51, 0.867, 0.409, 0.865, 0.811, 0.851, 0.882, 0.972, 0.821, 0.933, 0.723, 0.837]
Epoch: 3
Train Loss: 0.241, Accuracy of 12 AU classes: [0.244, 0.622, 0.644, 0.662, 0.652, 0.557, 0.717, 0.516, 0.633, 0.564, 0.655, 0.431]
Validation Loss: 0.201, Accuracy of 12 AU classes: [0.651, 0.881, 0.655, 0.891, 0.814, 0.853, 0.879, 0.986, 0.824, 0.936, 0.736, 0.89]
Epoch: 4
Train Loss: 0.23, Accuracy of 12 AU classes: [0.734, 0.699, 0.688, 0.69, 0.67, 0.594, 0.744, 0.629, 0.717, 0.666, 0.67, 0.536]
Validation Loss: 0.192, Accuracy of 12 AU cl

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_mlp_{viau}.pth'))
    mlp_model = MLPModel(num_classes = 12).to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, f1s, f1t, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, f1_threshold: {f1t}, accuracy: {acc}')

best metric: f1_score 0.502, f1_threshold: 0.258, accuracy: [0.914, 0.908, 0.856, 0.878, 0.837, 0.888, 0.888, 0.933, 0.824, 0.937, 0.764, 0.852]


##### EffNet + Vggish

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(AU_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.237, Accuracy of 12 AU classes: [0.831, 0.388, 0.805, 0.657, 0.687, 0.737, 0.774, 0.873, 0.539, 0.495, 0.702, 0.372]
Validation Loss: 0.19, Accuracy of 12 AU classes: [0.897, 0.593, 0.896, 0.875, 0.819, 0.889, 0.884, 0.989, 0.176, 0.342, 0.781, 0.812]
Epoch: 2
Train Loss: 0.216, Accuracy of 12 AU classes: [0.861, 0.625, 0.844, 0.726, 0.708, 0.757, 0.84, 0.904, 0.324, 0.277, 0.726, 0.63]
Validation Loss: 0.171, Accuracy of 12 AU classes: [0.915, 0.884, 0.819, 0.89, 0.845, 0.883, 0.895, 0.971, 0.446, 0.642, 0.795, 0.909]
Epoch: 3
Train Loss: 0.205, Accuracy of 12 AU classes: [0.873, 0.891, 0.858, 0.754, 0.718, 0.768, 0.846, 0.805, 0.51, 0.491, 0.739, 0.72]
Validation Loss: 0.164, Accuracy of 12 AU classes: [0.912, 0.915, 0.889, 0.877, 0.83, 0.877, 0.897, 0.987, 0.673, 0.941, 0.78, 0.911]
Epoch: 4
Train Loss: 0.197, Accuracy of 12 AU classes: [0.88, 0.906, 0.866, 0.771, 0.725, 0.775, 0.851, 0.838, 0.611, 0.323, 0.727, 0.767]
Validation Loss: 0.161, Accuracy of 12 AU

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_fusion_{viau}.pth'))
    AU_model = AU_fusion().to(device)
    AU_model.load_state_dict(AU_best_model)
    val_loss, f1s, f1t, acc = evaluate_model(AU_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, f1_threshold: {f1t}, accuracy: {acc}')

best metric: f1_score 0.57, f1_threshold: 0.258, accuracy: [0.931, 0.919, 0.862, 0.886, 0.833, 0.881, 0.896, 0.961, 0.74, 0.799, 0.782, 0.811]


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.299, Accuracy of 12 AU classes: [0.14, 0.205, 0.18, 0.493, 0.472, 0.39, 0.539, 0.043, 0.031, 0.045, 0.63, 0.12]
Validation Loss: 0.245, Accuracy of 12 AU classes: [0.284, 0.091, 0.51, 0.888, 0.7, 0.848, 0.886, 0.151, 0.176, 0.21, 0.736, 0.721]
Epoch: 2
Train Loss: 0.266, Accuracy of 12 AU classes: [0.126, 0.336, 0.5, 0.633, 0.59, 0.511, 0.686, 0.226, 0.099, 0.189, 0.63, 0.191]
Validation Loss: 0.212, Accuracy of 12 AU classes: [0.131, 0.908, 0.139, 0.879, 0.797, 0.841, 0.881, 0.988, 0.557, 0.829, 0.737, 0.788]
Epoch: 3
Train Loss: 0.248, Accuracy of 12 AU classes: [0.148, 0.541, 0.611, 0.679, 0.632, 0.569, 0.731, 0.474, 0.341, 0.444, 0.661, 0.419]
Validation Loss: 0.196, Accuracy of 12 AU classes: [0.472, 0.909, 0.397, 0.878, 0.818, 0.844, 0.878, 0.989, 0.721, 0.936, 0.746, 0.895]
Epoch: 4
Train Loss: 0.237, Accuracy of 12 AU classes: [0.684, 0.643, 0.664, 0.703, 0.656, 0.601, 0.754, 0.599, 0.488, 0.576, 0.676, 0.533]
Validation Loss: 0.187, Accuracy of 12 AU cla

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_mlp_{viau}.pth'))
    mlp_model = MLPModel(num_classes = 12).to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, f1s, f1t, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, f1_threshold: {f1t}, accuracy: {acc}')

best metric: f1_score 0.524, f1_threshold: 0.233, accuracy: [0.921, 0.922, 0.856, 0.883, 0.831, 0.89, 0.886, 0.973, 0.824, 0.936, 0.783, 0.873]


##### EffNet

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(AU_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.237, Accuracy of 12 AU classes: [0.747, 0.821, 0.8, 0.698, 0.657, 0.705, 0.8, 0.751, 0.048, 0.053, 0.686, 0.834]
Validation Loss: 0.197, Accuracy of 12 AU classes: [0.903, 0.921, 0.852, 0.891, 0.852, 0.891, 0.888, 0.728, 0.187, 0.759, 0.791, 0.696]
Epoch: 2
Train Loss: 0.217, Accuracy of 12 AU classes: [0.797, 0.885, 0.842, 0.748, 0.703, 0.728, 0.826, 0.604, 0.251, 0.261, 0.717, 0.865]
Validation Loss: 0.177, Accuracy of 12 AU classes: [0.893, 0.903, 0.862, 0.88, 0.859, 0.893, 0.902, 0.956, 0.381, 0.728, 0.804, 0.794]
Epoch: 3
Train Loss: 0.205, Accuracy of 12 AU classes: [0.88, 0.906, 0.859, 0.769, 0.723, 0.74, 0.837, 0.363, 0.411, 0.402, 0.734, 0.876]
Validation Loss: 0.17, Accuracy of 12 AU classes: [0.899, 0.934, 0.89, 0.888, 0.859, 0.893, 0.887, 0.812, 0.567, 0.776, 0.789, 0.817]
Epoch: 4
Train Loss: 0.197, Accuracy of 12 AU classes: [0.883, 0.917, 0.868, 0.781, 0.736, 0.748, 0.844, 0.483, 0.512, 0.493, 0.745, 0.882]
Validation Loss: 0.167, Accuracy of 12 AU

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_fusion_{viau}.pth'))
    AU_model = AU_fusion().to(device)
    AU_model.load_state_dict(AU_best_model)
    val_loss, f1s, f1t, acc = evaluate_model(AU_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, f1_threshold: {f1t}, accuracy: {acc}')

best metric: f1_score 0.542, f1_threshold: 0.25, accuracy: [0.897, 0.937, 0.888, 0.881, 0.853, 0.892, 0.892, 0.87, 0.734, 0.845, 0.782, 0.9]


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, f1s_best, f1t_best, accbest = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, auft, weights, viau)

Epoch: 1
Train Loss: 0.313, Accuracy of 12 AU classes: [0.281, 0.339, 0.204, 0.492, 0.504, 0.518, 0.513, 0.568, 0.064, 0.039, 0.627, 0.076]
Validation Loss: 0.259, Accuracy of 12 AU classes: [0.119, 0.286, 0.662, 0.891, 0.806, 0.843, 0.889, 0.877, 0.282, 0.072, 0.711, 0.089]
Epoch: 2
Train Loss: 0.276, Accuracy of 12 AU classes: [0.395, 0.226, 0.296, 0.503, 0.6, 0.498, 0.673, 0.436, 0.05, 0.253, 0.627, 0.086]
Validation Loss: 0.212, Accuracy of 12 AU classes: [0.882, 0.749, 0.833, 0.867, 0.818, 0.847, 0.878, 0.731, 0.194, 0.068, 0.721, 0.094]
Epoch: 3
Train Loss: 0.256, Accuracy of 12 AU classes: [0.558, 0.451, 0.466, 0.587, 0.635, 0.561, 0.636, 0.411, 0.266, 0.235, 0.635, 0.229]
Validation Loss: 0.195, Accuracy of 12 AU classes: [0.474, 0.903, 0.193, 0.872, 0.825, 0.849, 0.88, 0.908, 0.769, 0.898, 0.668, 0.616]
Epoch: 4
Train Loss: 0.244, Accuracy of 12 AU classes: [0.637, 0.574, 0.554, 0.631, 0.532, 0.596, 0.68, 0.544, 0.44, 0.416, 0.648, 0.369]
Validation Loss: 0.185, Accuracy of 12

In [ ]:
try:
    print(f'best metric: f1_score {f1s_best}, f1_threshold: {f1t_best}, accuracy: {accbest}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_mlp_{viau}.pth'))
    mlp_model = MLPModel(num_classes = 12).to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, f1s, f1t, acc = evaluate_model(mlp_model, val_loader, auft, weights)
    print(f'best metric: f1_score {f1s}, f1_threshold: {f1t}, accuracy: {acc}')

best metric: f1_score 0.513, f1_threshold: 0.217, accuracy: [0.923, 0.93, 0.858, 0.863, 0.844, 0.885, 0.888, 0.934, 0.825, 0.937, 0.772, 0.861]


### Testing

##### EffNet + Wav2vec2

In [ ]:
%%timeit -n 1 -r 1
print('AU_model')
print(visual_feat + ' & ' + auft)
AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_model/best_AU_fusion_{viau}_loss.pth'))
AU_model = AU_fusion().to(device)
AU_model.load_state_dict(AU_best_model)
test_loss, f1s, f1t, acc, threshold = evaluate_model(AU_model, test_loader, auft, weights1)
print(f'Test set: f1_score: {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

AU_model
visualfeat_enet_b2_8_best & audiofeat_wav2vec2
Test set: f1_score: 0.497, accuracy: [0.86, 0.889, 0.857, 0.79, 0.764, 0.8, 0.858, 0.901, 0.938, 0.932, 0.734, 0.803], f1_threshold: [0.2, 0.1, 0.30000000000000004, 0.30000000000000004, 0.5, 0.4, 0.4, 0.1, 0.1, 0.1, 0.4, 0.1]
1min 29s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_model/best_AU_mlp_{viau}.pth'))
mlp_model = MLPModel(num_classes = 12).to(device)
mlp_model.load_state_dict(mlp_best_model)
test_loss, f1s, f1t, acc, threshold = evaluate_model(mlp_model, test_loader, visual_feat, auft, weights)
print(f'Test set: f1_score: {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

MLP_model
visualfeat_enet_b2_8_best & audiofeat_wav2vec2
Test set: f1_score: 0.513, accuracy: [0.875, 0.913, 0.858, 0.813, 0.757, 0.807, 0.863, 0.879, 0.954, 0.926, 0.746, 0.849], f1_threshold: [0.7000000000000001, 0.8, 0.7000000000000001, 0.6, 0.5, 0.6, 0.7000000000000001, 0.6, 0.8, 0.8, 0.30000000000000004, 0.7000000000000001]
50.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##### EffNet + Vggish

In [ ]:
%%timeit -n 1 -r 1
print('AU_model')
print(visual_feat + ' & ' + auft)
AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_fusion_{viau}.pth'))
AU_model = AU_fusion().to(device)
AU_model.load_state_dict(AU_best_model)
test_loss, f1s, f1t, acc, threshold = evaluate_model(AU_model, test_loader, auft, weights1)
print(f'Test set: f1_score: {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

AU_model
visualfeat_enet_b2_8_best & audiofeat_vggish
Test set: f1_score: 0.494, accuracy: [0.886, 0.914, 0.86, 0.79, 0.772, 0.805, 0.867, 0.907, 0.925, 0.935, 0.76, 0.88], f1_threshold: [0.30000000000000004, 0.2, 0.30000000000000004, 0.4, 0.5, 0.4, 0.5, 0.1, 0.1, 0.2, 0.5, 0.2]
1min 26s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_model/best_AU_mlp_{viau}.pth'))
mlp_model = MLPModel(num_classes = 12).to(device)
mlp_model.load_state_dict(mlp_best_model)
test_loss, f1s, f1t, acc, threshold = evaluate_model(mlp_model, test_loader, visual_feat, auft, weights)
print(f'Test set: f1_score: {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

MLP_model
visualfeat_enet_b2_8_best & audiofeat_vggish
Test set: f1_score: 0.52, accuracy: [0.875, 0.913, 0.849, 0.809, 0.773, 0.8, 0.865, 0.897, 0.932, 0.922, 0.759, 0.849], f1_threshold: [0.7000000000000001, 0.8, 0.6, 0.6, 0.5, 0.5, 0.7000000000000001, 0.7000000000000001, 0.7000000000000001, 0.8, 0.30000000000000004, 0.7000000000000001]
48.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##### EffNet

In [ ]:
%%timeit -n 1 -r 1
print('AU_model')
print(visual_feat + ' & ' + auft)
AU_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_model/best_AU_fusion_{viau}.pth'))
AU_model = AU_fusion().to(device)
AU_model.load_state_dict(AU_best_model)
test_loss, f1s, f1t, acc, threshold = evaluate_model(AU_model, test_loader, auft, weights)
print(f'Test set: f1_score: {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

AU_model
visualfeat_enet_b2_8_best & nope
Test set: f1_score: 0.509, accuracy: [0.868, 0.911, 0.854, 0.794, 0.765, 0.793, 0.853, 0.905, 0.943, 0.951, 0.742, 0.838], f1_threshold: [0.7000000000000001, 0.8, 0.5, 0.5, 0.5, 0.5, 0.6, 0.5, 0.7000000000000001, 0.8, 0.30000000000000004, 0.6]
1min 28s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_mlp_{viau}_f1s.pth'))
mlp_model = MLPModel(num_classes = 12).to(device)
mlp_model.load_state_dict(mlp_best_model)
test_loss, f1s, f1t, acc, threshold = evaluate_model(mlp_model, test_loader, visual_feat, auft, weights)
print(f'Test set: f1_score: {f1s}, accuracy: {acc}, f1_threshold: {threshold}')

MLP_model
visualfeat_enet_b2_8_best & nope
Test set: f1_score: 0.521, accuracy: [0.878, 0.916, 0.864, 0.791, 0.772, 0.795, 0.86, 0.865, 0.933, 0.919, 0.757, 0.846], f1_threshold: [0.7000000000000001, 0.8, 0.7000000000000001, 0.5, 0.5, 0.5, 0.6, 0.6, 0.7000000000000001, 0.8, 0.30000000000000004, 0.7000000000000001]
52.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Ensembling

#### EffNet_B2 + Wav2Vec2

In [ ]:
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_model/best_AU_mlp_{viau}.pth'))
mlp_model_wav = MLPModel().to(device)
mlp_model_wav.load_state_dict(mlp_best_model)

<All keys matched successfully>

In [ ]:
iterator = iter(test_loader)
all_preds_wav, all_targets_wav = [], []
img = []
for AU in iterator:
    frame = AU['frame']
    for imgname in frame:
        img.append(imgname)
    vis_feat, aud_feat, y = AU[visual_feat], AU[auft], AU['label']
    vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
    pred, au_pred = mlp_model_wav(vis_feat, aud_feat)
    all_preds_wav.extend(au_pred.cpu().tolist())
    all_targets_wav.extend(y.cpu().tolist())

#### EffNet_B2 + Vggish

In [ ]:
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_AU_model/best_AU_mlp_{viau}.pth'))
mlp_model = MLPModel().to(device)
mlp_model.load_state_dict(mlp_best_model)

<All keys matched successfully>

In [ ]:
iterator = iter(test_loader)
all_preds, all_targets = [], []
img1 = []
for AU in iterator:
    frame = AU['frame']
    for imgname in frame:
        img1.append(imgname)
    vis_feat, aud_feat, y = AU[visual_feat], AU[auft], AU['label']
    vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
    pred, au_pred = mlp_model(vis_feat, aud_feat)
    all_preds.extend(au_pred.cpu().tolist())
    all_targets.extend(y.cpu().tolist())

#### EffNet_B0 + Vggish

In [ ]:
mlp_best_model_b0 = torch.load(os.path.join(root,f'models/ABAW6/{vis1}/best_AU_mlp_{viau}.pth'))
mlp_model_b0 = MLPModel_b0(num_classes = 12).to(device)
mlp_model_b0.load_state_dict(mlp_best_model_b0)

<All keys matched successfully>

In [ ]:
iterator = iter(test_loader_b0)
all_preds_b0, all_targets_b0 = [], []
img = []
for AU in iterator:
    frame = AU['frame']
    for imgname in frame:
        img.append(imgname)
    vis_feat, aud_feat, y = AU[visual_feat_1], AU[auft], AU['label']
    vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
    pred, au_pred = mlp_model_b0(vis_feat, aud_feat)
    all_preds_b0.extend(au_pred.cpu().tolist())
    all_targets_b0.extend(y.cpu().tolist())

#### Ensemble

##### EffNet_B2 + Wav2Vec2 + Vggish

In [ ]:
zipped_data = zip(img1, all_preds, all_targets)
zipped_data_wav = zip(img, all_preds_wav, all_targets_wav)

sorted_data = sorted(zipped_data)
sorted_data_wav = sorted(zipped_data_wav)

re_img_wav, re_all_preds_wav, re_all_targets_wav = zip(*sorted_data_wav)
re_img, re_all_preds, re_all_targets = zip(*sorted_data)

In [ ]:
f1s, _, _, thresholds = compute_AU_F1(re_all_preds, re_all_targets_wav)
round(f1s,3), thresholds

(0.52,
 [0.7000000000000001,
  0.8,
  0.6,
  0.6,
  0.5,
  0.5,
  0.7000000000000001,
  0.7000000000000001,
  0.7000000000000001,
  0.8,
  0.30000000000000004,
  0.7000000000000001])

In [ ]:
for w in np.linspace(0, 1, 11):
    print(f'{w = }')
    w = np.array([w])
    y_ensemble = w * re_all_preds + (1 - w) * re_all_preds_wav
    new_pred = (y_ensemble >= thresholds).astype(int)
    f1_scores, f1_thresh, acc, threshold = compute_AU_F1(re_all_targets, new_pred,thresh=np.arange(0.1,1,0.1))
    print(f'f1 score: {f1_scores:.3f}, accuracy: {acc}')

w = 0.0
f1 score: 0.512, accuracy: [0.875, 0.913, 0.839, 0.813, 0.757, 0.787, 0.863, 0.913, 0.922, 0.926, 0.746, 0.849]
w = 0.1
f1 score: 0.515, accuracy: [0.877, 0.914, 0.843, 0.814, 0.759, 0.79, 0.865, 0.914, 0.926, 0.929, 0.747, 0.851]
w = 0.2
f1 score: 0.517, accuracy: [0.878, 0.915, 0.846, 0.815, 0.762, 0.792, 0.865, 0.915, 0.929, 0.931, 0.749, 0.853]
w = 0.30000000000000004
f1 score: 0.520, accuracy: [0.88, 0.916, 0.848, 0.816, 0.764, 0.794, 0.866, 0.915, 0.932, 0.931, 0.751, 0.854]
w = 0.4
f1 score: 0.521, accuracy: [0.88, 0.917, 0.85, 0.816, 0.766, 0.797, 0.866, 0.915, 0.934, 0.932, 0.752, 0.855]
w = 0.5
f1 score: 0.522, accuracy: [0.88, 0.917, 0.851, 0.816, 0.767, 0.798, 0.866, 0.914, 0.935, 0.931, 0.754, 0.855]
w = 0.6000000000000001
f1 score: 0.523, accuracy: [0.88, 0.917, 0.852, 0.815, 0.769, 0.799, 0.867, 0.912, 0.936, 0.93, 0.755, 0.855]
w = 0.7000000000000001
f1 score: 0.523, accuracy: [0.88, 0.916, 0.852, 0.814, 0.77, 0.8, 0.866, 0.909, 0.936, 0.929, 0.756, 0.854]
w = 0

In [ ]:
weight=np.array([0.6])
y_ensemble = weight * re_all_preds + (1 - weight) * re_all_preds_wav
new_pred = ((y_ensemble >= thresholds) * 1)
f1_scores, f1_thresh, acc, threshold = compute_AU_F1(re_all_targets, new_pred,thresh=np.arange(0.1,1,0.1))
print(f'f1 score: {f1_scores:.3f}, accuracy: {acc}, threshold: {threshold}')

f1 score: 0.523, accuracy: [0.88, 0.917, 0.852, 0.815, 0.769, 0.799, 0.867, 0.912, 0.936, 0.93, 0.755, 0.855], threshold: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


##### EffNet_B2 + EffNet_B0 + Vggish

In [ ]:
zipped_data = zip(img1, all_preds, all_targets)
zipped_data_b0 = zip(img, all_preds_b0, all_targets_b0)

sorted_data = sorted(zipped_data)
sorted_data_b0 = sorted(zipped_data_b0)

re_img_b0, re_all_preds_b0, re_all_targets_b0 = zip(*sorted_data_b0)
re_img, re_all_preds, re_all_targets = zip(*sorted_data)

In [ ]:
f1s, _, _, thresholds = compute_AU_F1(re_all_preds_b0, re_all_targets)
round(f1s,3), thresholds

(0.52,
 [0.8,
  0.9,
  0.6,
  0.5,
  0.5,
  0.6,
  0.7000000000000001,
  0.7000000000000001,
  0.7000000000000001,
  0.8,
  0.30000000000000004,
  0.7000000000000001])

In [ ]:
for w in np.linspace(0, 1, 11):
    print(f'{w = }')
    w = np.array([w])
    y_ensemble = w * re_all_preds + (1 - w) * re_all_preds_b0
    new_pred = (y_ensemble >= thresholds).astype(int)
    f1_scores, f1_thresh, acc, threshold = compute_AU_F1(re_all_targets, new_pred,thresh=np.arange(0.1,1,0.1))
    print(f'f1 score: {f1_scores:.3f}, accuracy: {acc}')

w = 0.0
f1 score: 0.520, accuracy: [0.877, 0.915, 0.853, 0.784, 0.78, 0.807, 0.868, 0.932, 0.948, 0.933, 0.777, 0.849], threshold: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
w = 0.1
f1 score: 0.524, accuracy: [0.883, 0.92, 0.857, 0.787, 0.783, 0.811, 0.87, 0.934, 0.951, 0.935, 0.778, 0.853], threshold: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
w = 0.2
f1 score: 0.528, accuracy: [0.888, 0.923, 0.86, 0.79, 0.786, 0.815, 0.873, 0.936, 0.954, 0.936, 0.779, 0.857], threshold: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
w = 0.30000000000000004
f1 score: 0.531, accuracy: [0.892, 0.925, 0.864, 0.794, 0.788, 0.818, 0.875, 0.938, 0.956, 0.937, 0.778, 0.86], threshold: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
w = 0.4
f1 score: 0.532, accuracy: [0.894, 0.926, 0.867, 0.797, 0.79, 0.821, 0.876, 0.938, 0.958, 0.937, 0.778, 0.863], threshold: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
w = 0.5
f1 score: 0.533, a

In [ ]:
weight=np.array([0.5])
y_ensemble = weight * re_all_preds + (1 - weight) * re_all_preds_b0
new_pred = ((y_ensemble >= thresholds) * 1)
f1_scores, f1_thresh, acc, threshold = compute_AU_F1(re_all_targets, new_pred,thresh=np.arange(0.1,1,0.1))
print(f'f1 score: {f1_scores:.3f}, accuracy: {acc}, threshold: {threshold}')

f1 score: 0.533, accuracy: [0.895, 0.926, 0.868, 0.8, 0.791, 0.823, 0.876, 0.938, 0.96, 0.937, 0.777, 0.864], threshold: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


### Smoothing

In [ ]:
tsk = task[1]
vis = vis_typ[0]
viau = vis_aud[1]
auft = audio_feat[1]

In [ ]:
index, pred, lab = [], [], []
test_vid = {}
for i, val in enumerate(re_img):
    ind = int(val.split('/')[1][:-4])
    vname = val.split('/')[0]
    if i == 0:
        prename = vname
        index.append(ind)
        pred.append(new_pred[i])
        lab.append(re_all_targets[i])
    else:
        if vname == prename:
            index.append(ind)
            pred.append(new_pred[i])
            lab.append(re_all_targets[i])
        else:
            combined = list(zip(index, pred, lab))
            combined_sorted = sorted(combined, key=lambda x: x[0])
            index_list_sorted, pred_list_sorted, lab_list_sorted = zip(*combined_sorted)
            test_vid[prename] = (list(index_list_sorted), list(pred_list_sorted), list(lab_list_sorted))
            prename = vname
            index, pred, lab = [], [], []
            index.append(ind)
            pred.append(new_pred[i])
            lab.append(re_all_targets[i])

##### EffNet_B2 + Wav2Vec2 + Vggish

In [ ]:
thresholds = np.array([0.7000000000000001,
  0.8,
  0.6,
  0.6,
  0.5,
  0.5,
  0.7000000000000001,
  0.7000000000000001,
  0.7000000000000001,
  0.8,
  0.30000000000000004,
  0.7000000000000001])

In [ ]:
hyperparams=[(isMean,delta) for delta in [0, 5, 15, 30, 60, 100, 200] for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
timing_results = {(isMean, delta): 0 for isMean, delta in hyperparams}
for videoname,(img, predict, label) in test_vid.items():
    for i,ind in enumerate(img):
        total_true.append(label[i])
    preds_proba = smooth_prediction(img, predict)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        preds=[]
        start = time.time()
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            if isMean:
                _, proba = slide_window(preds_proba, i, delta, 'mean')
            else:
                _, proba = slide_window(preds_proba, i, delta, 'median')
            aus = (proba>=thresholds)*1
            preds.append(aus)
        for i,ind in enumerate(img):
            if label[i][0]>=-1 and label[i][1]>=-1:
                total_preds[hInd].append(preds[ind-1])
        end = time.time()
        timing_results[(isMean, delta)] += end - start
total_true=np.array(total_true)

In [ ]:
for hInd, (isMean, delta) in enumerate(hyperparams):
    preds = np.array(total_preds[hInd])
    accuracy = round((preds == total_true).mean(), 3)
    f1 = round(np.mean([f1_score(y_true=total_true[:,i],y_pred=preds[:,i]) for i in range(preds.shape[1])]), 3)
    mean_or_median = 'mean' if isMean else 'median'
    time_taken = round(timing_results[(isMean, delta)],3)
    print(f'{mean_or_median}; delta: {delta}; Acc: {accuracy}; F1: {f1}; Time: {time_taken}s')

mean; delta: 0; Acc: 0.857; F1: 0.523; Time: 7.502s
mean; delta: 5; Acc: 0.869; F1: 0.525; Time: 7.965s
median; delta: 5; Acc: 0.863; F1: 0.532; Time: 16.839s
mean; delta: 15; Acc: 0.871; F1: 0.511; Time: 8.342s
median; delta: 15; Acc: 0.866; F1: 0.534; Time: 17.291s
mean; delta: 30; Acc: 0.87; F1: 0.492; Time: 8.718s
median; delta: 30; Acc: 0.867; F1: 0.528; Time: 17.746s
mean; delta: 60; Acc: 0.868; F1: 0.462; Time: 9.44s
median; delta: 60; Acc: 0.866; F1: 0.515; Time: 19.077s
mean; delta: 100; Acc: 0.864; F1: 0.431; Time: 11.532s
median; delta: 100; Acc: 0.864; F1: 0.502; Time: 21.379s
mean; delta: 200; Acc: 0.857; F1: 0.39; Time: 13.451s
median; delta: 200; Acc: 0.858; F1: 0.472; Time: 25.658s


##### EffNet_B2 + EffNet_B0 + Vggish

In [ ]:
thresholds = np.array([0.8,
  0.9,
  0.6,
  0.5,
  0.5,
  0.6,
  0.7000000000000001,
  0.7000000000000001,
  0.7000000000000001,
  0.8,
  0.30000000000000004,
  0.7000000000000001])

In [ ]:
hyperparams=[(isMean,delta) for delta in [0, 5, 15, 30, 60, 100, 200] for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
timing_results = {(isMean, delta): 0 for isMean, delta in hyperparams}
for videoname,(img, predict, label) in test_vid.items():
    for i,ind in enumerate(img):
        total_true.append(label[i])
    preds_proba = smooth_prediction(img, predict)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        preds=[]
        start = time.time()
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            if isMean:
                _, proba = slide_window(preds_proba, i, delta, 'mean')
            else:
                _, proba = slide_window(preds_proba, i, delta, 'median')
            aus = (proba>=thresholds)*1
            preds.append(aus)
        for i,ind in enumerate(img):
            if label[i][0]>=-1 and label[i][1]>=-1:
                total_preds[hInd].append(preds[ind-1])
        end = time.time()
        timing_results[(isMean, delta)] += end - start
total_true=np.array(total_true)

In [ ]:
for hInd, (isMean, delta) in enumerate(hyperparams):
    preds = np.array(total_preds[hInd])
    accuracy = round((preds == total_true).mean(), 3)
    f1 = round(np.mean([f1_score(y_true=total_true[:,i],y_pred=preds[:,i]) for i in range(preds.shape[1])]), 3)
    mean_or_median = 'mean' if isMean else 'median'
    time_taken = round(timing_results[(isMean, delta)],3)
    print(f'{mean_or_median}; delta: {delta}; Acc: {accuracy}; F1: {f1}; Time: {time_taken}s')

mean; delta: 0; Acc: 0.871; F1: 0.534; Time: 8.697s
mean; delta: 5; Acc: 0.878; F1: 0.524; Time: 9.522s
median; delta: 5; Acc: 0.875; F1: 0.54; Time: 22.173s
mean; delta: 15; Acc: 0.877; F1: 0.499; Time: 9.23s
median; delta: 15; Acc: 0.877; F1: 0.535; Time: 19.396s
mean; delta: 30; Acc: 0.876; F1: 0.472; Time: 9.172s
median; delta: 30; Acc: 0.877; F1: 0.524; Time: 19.412s
mean; delta: 60; Acc: 0.872; F1: 0.437; Time: 9.686s
median; delta: 60; Acc: 0.874; F1: 0.506; Time: 20.104s
mean; delta: 100; Acc: 0.867; F1: 0.41; Time: 10.824s
median; delta: 100; Acc: 0.871; F1: 0.489; Time: 22.654s
mean; delta: 200; Acc: 0.86; F1: 0.368; Time: 14.02s
median; delta: 200; Acc: 0.864; F1: 0.457; Time: 27.107s


## VA Estimation Challenge

### Loading data

In [ ]:
# Cropped_aligned images
vis = vis_typ[0]
vis1 = vis_typ[2]

In [ ]:
# Cropped images
vis = vis_typ[1]

#### Effnet + wav2vec2

In [ ]:
auft = audio_feat[0]
viau = vis_aud[0]

#### Effnet + vggish

In [ ]:
auft = audio_feat[1]
viau = vis_aud[1]

#### Effnet

In [ ]:
auft = audio_feat[2]
viau = vis_aud[2]

#### Train

In [ ]:
if auft == audio_feat[0]:
    with open(os.path.join(root,f'models/ABAW6/{vis}/VA/{task[2]}_{typ[0]}_visual.pkl'), 'rb') as f:
        data1 = pickle.load(f)
    with open(os.path.join(root, f'data/Annotations/{task[2]}/{typ[0]}.txt'), 'r') as f:
        vidnames = f.read().splitlines()
    task1 = task[2]
    feature_a = 'audiofeat_wav2vec2'
    feat_root = os.path.join(root + '/models/ABAW6', feature_a)
    filenames = os.listdir(feat_root)[:]
    for vname in tqdm(vidnames):
            feature = np.load(os.path.join(feat_root, f'{vname}.npy'), allow_pickle=True).tolist()
            for imgname, val in feature.items():
                if imgname in data1[task1][vname]:
                    data1[task1][vname][imgname].update({f'{feature_a}': val})
            for img, value in list(data1[task1][vname].items()):
                if len(value) < 3:
                    data1[task1][vname].pop(img)
elif auft == audio_feat[1]:
    with open(os.path.join(root,f'models/ABAW6/{vis}/VA/{task[2]}_{typ[0]}_visual.pkl'), 'rb') as f:
        data1 = pickle.load(f)
    with open(os.path.join(root, f'data/Annotations/{task[2]}/{typ[0]}.txt'), 'r') as f:
        vidnames = f.read().splitlines()
    task1 = task[2]
    feature_a = 'audiofeat_vggish'
    feat_root = os.path.join(root + '/models/ABAW6', feature_a)
    filenames = os.listdir(feat_root)[:]
    for vname in tqdm(vidnames):
            feature = np.load(os.path.join(feat_root, f'{vname}.npy'), allow_pickle=True).tolist()
            for imgname, val in feature.items():
                if imgname in data1[task1][vname]:
                    data1[task1][vname][imgname].update({f'{feature_a}': val})
            for img, value in list(data1[task1][vname].items()):
                if len(value) < 3:
                    data1[task1][vname].pop(img)
else:
    with open(os.path.join(root,f'models/ABAW6/{vis}/VA/{task[2]}_{typ[0]}_{viau}.pkl'), 'rb') as f:
        data1 = pickle.load(f)

100%|██████████| 285/285 [03:19<00:00,  1.43it/s]


In [ ]:
dims = 0
iname = []
task1 = task[2]
with open(os.path.join(root, f'data/Annotations/{task[2]}/{typ[0]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data1[task1][vname])
    for img in data1[task1][vname].keys():
        iname.append(img)

100%|██████████| 285/285 [00:00<00:00, 1425.58it/s]


In [ ]:
dataset = ABAW_dataset1(data1, iname, dims, task1)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0, drop_last=True)

#### Val

In [ ]:
with open(os.path.join(root,f'models/ABAW6/{vis}/VA/{task[2]}_{typ[1]}_{viau}.pkl'), 'rb') as f:
    data2 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[2]
with open(os.path.join(root, f'data/Annotations/{task[2]}/{typ[1]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data2[task1][vname])
    for img in data2[task1][vname].keys():
        iname.append(img)

100%|██████████| 71/71 [00:00<00:00, 1084.70it/s]


In [ ]:
dataset = ABAW_dataset1(data2, iname, dims, task1)
val_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, drop_last=False)

#### Test

In [ ]:
with open(os.path.join(root,f'models/ABAW6/{vis}/VA/{task[2]}_{typ[2]}_{viau}.pkl'), 'rb') as f:
    data3 = pickle.load(f)

In [ ]:
dims = 0
iname = []
task1 = task[2]
with open(os.path.join(root, f'data/Annotations/{task[2]}/{typ[2]}.txt'), 'r') as f:
    vidnames = f.read().splitlines()
for vname in tqdm(vidnames):
    dims += len(data3[task1][vname])
    for img in data3[task1][vname].keys():
        iname.append(img)

100%|██████████| 76/76 [00:00<00:00, 1175.35it/s]


In [ ]:
dataset = ABAW_dataset1(data3, iname, dims, task1)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, drop_last=False)

### Modeling

In [ ]:
class VA_fusion(nn.Module):
    def __init__(self, batchsize = batch_size, audio_ft = auft, hidden_size = [512, 128, batch_size]):
        super(VA_fusion, self).__init__()
        self.batchsize = batchsize
        if audio_ft == 'audiofeat_wav2vec2':
            self.concat_dim = 2176    #1408+768
        elif audio_ft == 'audiofeat_vggish':
            self.concat_dim = 1536    #1408+128
        elif audio_ft == 'nope':
            self.concat_dim = 1408    #visual only
        self.hidden_size = hidden_size
        self.feat_fc = nn.Conv1d(self.concat_dim, hidden_size[0], 1, padding=0)
        self.activ = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(p=0.3)
        self.conv1 = nn.Conv1d(hidden_size[0], hidden_size[1], 1, padding=0)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size[1], nhead=4, dim_feedforward=hidden_size[1], dropout=0.3)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)
        self.vhead = nn.Sequential(
                nn.Linear(hidden_size[1], hidden_size[2]),
                nn.BatchNorm1d(hidden_size[2]),
                nn.Linear(hidden_size[2], 1),
                )
        self.ahead = nn.Sequential(
                nn.Linear(hidden_size[1], hidden_size[2]),
                nn.BatchNorm1d(hidden_size[2]),
                nn.Linear(hidden_size[2], 1),
                )

    def forward(self, vis_feat, aud_feat):
        if aud_feat == None:
            feat = vis_feat
        else:
            inputs = [vis_feat]
            inputs.append(aud_feat)
            feat = torch.cat(inputs,dim=1)
        feat = torch.transpose(feat,0,1)
        feat = self.feat_fc(feat)
        feat = self.activ(feat)
        out = self.conv1(feat)
        out = torch.transpose(out,0,1)
        out = self.transformer_encoder(out)
        vout = self.vhead(out)
        aout = self.ahead(out)

        return vout, aout, torch.tanh(vout), torch.tanh(aout)

In [ ]:
VA_model = VA_fusion().to(device)
VA_model

VA_fusion(
  (feat_fc): Conv1d(2176, 512, kernel_size=(1,), stride=(1,))
  (activ): LeakyReLU(negative_slope=0.1)
  (dropout): Dropout(p=0.3, inplace=False)
  (conv1): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=128, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
    )
  )
  (vhead): Sequential(
    (0): Linear(in_features=128, out_feature

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, audio_ft = auft, num_classes=1):
        super(MLPModel, self).__init__()
        if audio_ft == 'audiofeat_wav2vec2':
            self.concat_dim = 2176    #1408+768
        elif audio_ft == 'audiofeat_vggish':
            self.concat_dim = 1536    #1408+128
        elif audio_ft == 'nope':
            self.concat_dim = 1408    #visual only
        self.feat_fc = nn.Conv1d(self.concat_dim, 512, 1, padding=0)
        self.vhead = nn.Sequential(
            nn.Linear(512, 128),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(128),
            nn.Linear(128, num_classes)
        )

        self.ahead = nn.Sequential(
            nn.Linear(self.concat_dim, 128),
            nn.BatchNorm1d(128),
            nn.Linear(128, num_classes)
        )
    def forward(self, vis_feat, aud_feat):
        if aud_feat == None:
            feat = vis_feat
        else:
            inputs = [vis_feat]
            inputs.append(aud_feat)
            feat = torch.cat(inputs, dim=1)
        vfeat = self.feat_fc(torch.transpose(feat,0,1))
        vfeat = torch.transpose(vfeat,0,1)
        vout = self.vhead(vfeat)
        aout = self.ahead(feat)

        return vout, aout, torch.tanh(vout), torch.tanh(aout)

In [ ]:
mlp_model = MLPModel().to(device)
mlp_model

MLPModel(
  (feat_fc): Conv1d(1536, 512, kernel_size=(1,), stride=(1,))
  (vhead): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=128, out_features=1, bias=True)
  )
  (ahead): Sequential(
    (0): Linear(in_features=1536, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [ ]:
def evaluate_model(model, data_loader, au_feat):
    model.eval()
    total_loss = []
    all_targets = []
    all_preds = []
    mse = []
    with torch.no_grad():
        iterator = iter(data_loader)
        for i in range(len(data_loader)):
            VA = next(iterator)
            if au_feat == 'nope':
                vis_feat, y = VA[visual_feat], VA['label']
                vis_feat, y = vis_feat.to(device), y.to(device)
                aud_feat = None
            else:
                vis_feat, aud_feat, y = VA[visual_feat], VA[au_feat], VA['label']
                vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
            Vpred, Apred, v_pred, a_pred = model(vis_feat, aud_feat)
            mse_loss, ccc_loss = compute_VA_loss(Vpred, Apred, y)
            total_loss.append(ccc_loss.item())
            mse.append(mse_loss.item())
            preds = torch.cat((v_pred, a_pred), dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_targets.extend(y.cpu().tolist())

    ccc1, ccc2 = compute_VA_CCC(all_preds, all_targets)
    return round(np.mean(total_loss),3), round(np.mean(mse),3), round(ccc1,3), round(ccc2,3)

#### Utils

In [ ]:
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, VA_model.parameters()), lr=0.00005, betas=(0.9, 0.999), weight_decay=0.00001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=1e-5)

In [ ]:
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, mlp_model.parameters()), lr=0.0005, betas=(0.9, 0.999), weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=1e-5)

### Training

In [ ]:
def train(model, mod_type, train_loader, val_loader, epoch, batch_size, optim, scheduler, au_feat, vis_aud):

    model.train(True)
    model.eval()
    loss_value = []
    loss_train = []
    loss_val = []
    loss_mse = []
    cc1best, cc2best = 0, 0

    for e in range(epoch):
        print(f'Training Epoch: {e+1}')
        iterator = iter(train_loader)
        for VA in iterator:
                if au_feat == 'nope':
                    vis_feat, y = VA[visual_feat_1], VA['label']
                    vis_feat, y = vis_feat.to(device), y.to(device)
                    aud_feat = None
                else:
                    vis_feat, aud_feat, y = VA[visual_feat_1], VA[au_feat], VA['label']
                    vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
                model.zero_grad()
                Vpred, Apred, v_pred, a_pred = model(vis_feat, aud_feat)
                mse_loss, ccc_loss = compute_VA_loss(Vpred, Apred, y)
                ccc_loss.backward()
                optim.step()
                loss_value.append(ccc_loss.item())
                loss_mse.append(mse_loss.item())
                preds = torch.cat((v_pred, a_pred), dim=1)

        avg_loss = round(np.mean(loss_value),3)
        loss_train.append(avg_loss)
        print(f'Train Loss: {avg_loss}, mse: {round(np.mean(loss_mse),3)}')

        val_loss, mse, ccc1, ccc2 = evaluate_model(model, val_loader, au_feat)
        loss_val.append(val_loss)

        if ccc1 > cc1best:
            torch.save(model.state_dict(), os.path.join(root,f'models/ABAW6/{vis}/best_VA_{mod_type}_{vis_aud}.pth'))
            cc1best = ccc1
            cc2best = ccc2
            best_loss = val_loss
            best_mse = mse

        print(f'Validation Loss: {val_loss}, mse: {mse}')

        scheduler.step(val_loss)
    return loss_train, loss_val, best_loss, best_mse, cc1best, cc2best

In [ ]:
def evaluate_model(model, data_loader, au_feat):
    model.eval()
    total_loss = []
    all_targets = []
    all_preds = []
    mse = []
    with torch.no_grad():
        iterator = iter(data_loader)
        for i in range(len(data_loader)):
            VA = next(iterator)
            if au_feat == 'nope':
                vis_feat, y = VA[visual_feat_1], VA['label']
                vis_feat, y = vis_feat.to(device), y.to(device)
                aud_feat = None
            else:
                vis_feat, aud_feat, y = VA[visual_feat_1], VA[au_feat], VA['label']
                vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
            Vpred, Apred, v_pred, a_pred = model(vis_feat, aud_feat)
            mse_loss, ccc_loss = compute_VA_loss(Vpred, Apred, y)
            total_loss.append(ccc_loss.item())
            mse.append(mse_loss.item())
            preds = torch.cat((v_pred, a_pred), dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_targets.extend(y.cpu().tolist())

    ccc1, ccc2 = compute_VA_CCC(all_preds, all_targets)
    return round(np.mean(total_loss),3), round(np.mean(mse),3), round(ccc1,3), round(ccc2,3)

#### Cropped_aligned images

##### Effnet + Wav2vec

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(VA_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 0.405, mse: 0.482
Validation Loss: 1.42, mse: 0.35
Training Epoch: 2
Train Loss: 0.329, mse: 0.48
Validation Loss: 1.42, mse: 0.35
Training Epoch: 3
Train Loss: 0.292, mse: 0.48
Validation Loss: 1.461, mse: 0.329
Training Epoch: 4
Train Loss: 0.269, mse: 0.479
Validation Loss: 1.464, mse: 0.339
Training Epoch: 5
Train Loss: 0.252, mse: 0.479
Validation Loss: 1.491, mse: 0.337
Training Epoch: 6
Train Loss: 0.239, mse: 0.479
Validation Loss: 1.473, mse: 0.334
Training Epoch: 7
Train Loss: 0.228, mse: 0.479
Validation Loss: 1.481, mse: 0.339
Training Epoch: 8
Train Loss: 0.218, mse: 0.479
Validation Loss: 1.479, mse: 0.343
Training Epoch: 9
Train Loss: 0.21, mse: 0.478
Validation Loss: 1.5, mse: 0.346
Training Epoch: 10
Train Loss: 0.203, mse: 0.478
Validation Loss: 1.467, mse: 0.347
1h 48min 44s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_fusion_{viau}.pth'))
    VA_model = VA_fusion().to(device)
    VA_model.load_state_dict(VA_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.352, CCC_Arousal: 0.48


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 0.522, mse: 0.484
Validation Loss: 1.396, mse: 0.338
Training Epoch: 2
Train Loss: 0.473, mse: 0.481
Validation Loss: 1.424, mse: 0.346
Training Epoch: 3
Train Loss: 0.452, mse: 0.48
Validation Loss: 1.426, mse: 0.346
Training Epoch: 4
Train Loss: 0.44, mse: 0.48
Validation Loss: 1.412, mse: 0.34
Training Epoch: 5
Train Loss: 0.432, mse: 0.48
Validation Loss: 1.433, mse: 0.342
Training Epoch: 6
Train Loss: 0.426, mse: 0.48
Validation Loss: 1.44, mse: 0.338
Training Epoch: 7
Train Loss: 0.42, mse: 0.479
Validation Loss: 1.435, mse: 0.339
Training Epoch: 8
Train Loss: 0.416, mse: 0.479
Validation Loss: 1.432, mse: 0.338
Training Epoch: 9
Train Loss: 0.412, mse: 0.479
Validation Loss: 1.432, mse: 0.346
Training Epoch: 10
Train Loss: 0.409, mse: 0.479
Validation Loss: 1.434, mse: 0.338
45min 37s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_mlp_{viau}_mse.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.38, CCC_Arousal: 0.445


##### Effnet + Vggish

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(VA_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 0.422, mse: 0.482
Validation Loss: 1.354, mse: 0.335
Training Epoch: 2
Train Loss: 0.331, mse: 0.48
Validation Loss: 1.393, mse: 0.35
Training Epoch: 3
Train Loss: 0.286, mse: 0.48
Validation Loss: 1.423, mse: 0.336
Training Epoch: 4
Train Loss: 0.256, mse: 0.479
Validation Loss: 1.443, mse: 0.343
Training Epoch: 5
Train Loss: 0.234, mse: 0.479
Validation Loss: 1.447, mse: 0.345
Training Epoch: 6
Train Loss: 0.217, mse: 0.479
Validation Loss: 1.43, mse: 0.345
Training Epoch: 7
Train Loss: 0.204, mse: 0.479
Validation Loss: 1.465, mse: 0.342
Training Epoch: 8
Train Loss: 0.192, mse: 0.479
Validation Loss: 1.467, mse: 0.333
Training Epoch: 9
Train Loss: 0.182, mse: 0.479
Validation Loss: 1.49, mse: 0.335
Training Epoch: 10
Train Loss: 0.173, mse: 0.478
Validation Loss: 1.466, mse: 0.341
1h 38min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_fusion_{viau}.pth'))
    VA_model = VA_fusion().to(device)
    VA_model.load_state_dict(VA_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.367, CCC_Arousal: 0.381


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 1.218, mse: 0.461
Validation Loss: 1.176, mse: 0.29
Training Epoch: 2
Train Loss: 1.178, mse: 0.468
Validation Loss: 1.183, mse: 0.308
Training Epoch: 3
Train Loss: 1.157, mse: 0.469
Validation Loss: 1.195, mse: 0.315
Training Epoch: 4
Train Loss: 1.144, mse: 0.469
Validation Loss: 1.174, mse: 0.308
Training Epoch: 5
Train Loss: 1.135, mse: 0.471
Validation Loss: 1.216, mse: 0.335
Training Epoch: 6
Train Loss: 1.126, mse: 0.471
Validation Loss: 1.234, mse: 0.341
Training Epoch: 7
Train Loss: 1.12, mse: 0.472
Validation Loss: 1.232, mse: 0.341
Training Epoch: 8
Train Loss: 1.113, mse: 0.473
Validation Loss: 1.212, mse: 0.352
Training Epoch: 9
Train Loss: 1.107, mse: 0.474
Validation Loss: 1.218, mse: 0.352
Training Epoch: 10
Train Loss: 1.102, mse: 0.475
Validation Loss: 1.222, mse: 0.37
1min 17s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}, Mean CCC: {round((ccc1+ccc2)/2,3)}')

best metric: CCC_Valence 0.412, CCC_Arousal: 0.486, Mean CCC: 0.449


##### Effnet

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(VA_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 0.458, mse: 0.483
Validation Loss: 1.45, mse: 0.276
Training Epoch: 2
Train Loss: 0.377, mse: 0.481
Validation Loss: 1.492, mse: 0.274
Training Epoch: 3
Train Loss: 0.338, mse: 0.48
Validation Loss: 1.537, mse: 0.286
Training Epoch: 4
Train Loss: 0.312, mse: 0.48
Validation Loss: 1.532, mse: 0.256
Training Epoch: 5
Train Loss: 0.294, mse: 0.48
Validation Loss: 1.564, mse: 0.261
Training Epoch: 6
Train Loss: 0.279, mse: 0.479
Validation Loss: 1.636, mse: 0.27
Training Epoch: 7
Train Loss: 0.266, mse: 0.479
Validation Loss: 1.606, mse: 0.29
Training Epoch: 8
Train Loss: 0.256, mse: 0.479
Validation Loss: 1.503, mse: 0.277
Training Epoch: 9
Train Loss: 0.248, mse: 0.479
Validation Loss: 1.628, mse: 0.275
Training Epoch: 10
Train Loss: 0.24, mse: 0.479
Validation Loss: 1.566, mse: 0.279
1h 27min 13s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_fusion_{viau}.pth'))
    VA_model = VA_fusion().to(device)
    VA_model.load_state_dict(VA_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.185, CCC_Arousal: 0.338


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, best_mse, cccv, ccca = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 1.281, mse: 0.467
Validation Loss: 1.254, mse: 0.284
Training Epoch: 2
Train Loss: 1.236, mse: 0.466
Validation Loss: 1.245, mse: 0.299
Training Epoch: 3
Train Loss: 1.214, mse: 0.468
Validation Loss: 1.24, mse: 0.313
Training Epoch: 4
Train Loss: 1.197, mse: 0.467
Validation Loss: 1.236, mse: 0.317
Training Epoch: 5
Train Loss: 1.184, mse: 0.469
Validation Loss: 1.231, mse: 0.303
Training Epoch: 6
Train Loss: 1.175, mse: 0.469
Validation Loss: 1.253, mse: 0.308
Training Epoch: 7
Train Loss: 1.169, mse: 0.469
Validation Loss: 1.249, mse: 0.306
Training Epoch: 8
Train Loss: 1.161, mse: 0.471
Validation Loss: 1.245, mse: 0.324
Training Epoch: 9
Train Loss: 1.155, mse: 0.472
Validation Loss: 1.243, mse: 0.325
Training Epoch: 10
Train Loss: 1.15, mse: 0.473
Validation Loss: 1.254, mse: 0.317
1min 9s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

#### Cropped images

##### Effnet + Wav2vec

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(VA_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 0.903, mse: 0.504
Validation Loss: 1.389, mse: 0.254
Training Epoch: 2
Train Loss: 0.793, mse: 0.494
Validation Loss: 1.377, mse: 0.243
Training Epoch: 3
Train Loss: 0.74, mse: 0.491
Validation Loss: 1.356, mse: 0.244
Training Epoch: 4
Train Loss: 0.704, mse: 0.49
Validation Loss: 1.324, mse: 0.232
Training Epoch: 5
Train Loss: 0.678, mse: 0.489
Validation Loss: 1.32, mse: 0.206
Training Epoch: 6
Train Loss: 0.657, mse: 0.488
Validation Loss: 1.384, mse: 0.236
Training Epoch: 7
Train Loss: 0.639, mse: 0.488
Validation Loss: 1.384, mse: 0.236
Training Epoch: 8
Train Loss: 0.624, mse: 0.488
Validation Loss: 1.329, mse: 0.22
Training Epoch: 9
Train Loss: 0.611, mse: 0.487
Validation Loss: 1.31, mse: 0.253
Training Epoch: 10
Train Loss: 0.599, mse: 0.487
Validation Loss: 1.295, mse: 0.228
3min 35s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_fusion_{viau}.pth'))
    VA_model = VA_fusion().to(device)
    VA_model.load_state_dict(VA_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.329, CCC_Arousal: 0.39


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 1.279, mse: 0.464
Validation Loss: 1.261, mse: 0.315
Training Epoch: 2
Train Loss: 1.231, mse: 0.47
Validation Loss: 1.242, mse: 0.302
Training Epoch: 3
Train Loss: 1.209, mse: 0.472
Validation Loss: 1.202, mse: 0.277
Training Epoch: 4
Train Loss: 1.192, mse: 0.473
Validation Loss: 1.218, mse: 0.275
Training Epoch: 5
Train Loss: 1.179, mse: 0.473
Validation Loss: 1.178, mse: 0.264
Training Epoch: 6
Train Loss: 1.166, mse: 0.474
Validation Loss: 1.139, mse: 0.254
Training Epoch: 7
Train Loss: 1.158, mse: 0.475
Validation Loss: 1.163, mse: 0.262
Training Epoch: 8
Train Loss: 1.15, mse: 0.476
Validation Loss: 1.147, mse: 0.263
Training Epoch: 9
Train Loss: 1.144, mse: 0.477
Validation Loss: 1.153, mse: 0.265
Training Epoch: 10
Train Loss: 1.137, mse: 0.477
Validation Loss: 1.161, mse: 0.257
1min 33s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.501, CCC_Arousal: 0.523


##### Effnet + Vggish

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(VA_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 0.904, mse: 0.484
Validation Loss: 1.319, mse: 0.281
Training Epoch: 2
Train Loss: 0.808, mse: 0.483
Validation Loss: 1.338, mse: 0.283
Training Epoch: 3
Train Loss: 0.753, mse: 0.482
Validation Loss: 1.357, mse: 0.28
Training Epoch: 4
Train Loss: 0.712, mse: 0.482
Validation Loss: 1.367, mse: 0.275
Training Epoch: 5
Train Loss: 0.678, mse: 0.481
Validation Loss: 1.375, mse: 0.271
Training Epoch: 6
Train Loss: 0.649, mse: 0.481
Validation Loss: 1.383, mse: 0.269
Training Epoch: 7
Train Loss: 0.622, mse: 0.481
Validation Loss: 1.4, mse: 0.269
Training Epoch: 8
Train Loss: 0.599, mse: 0.48
Validation Loss: 1.417, mse: 0.269
Training Epoch: 9
Train Loss: 0.577, mse: 0.48
Validation Loss: 1.434, mse: 0.273
Training Epoch: 10
Train Loss: 0.556, mse: 0.48
Validation Loss: 1.445, mse: 0.272
3min 7s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_fusion_{viau}.pth'))
    VA_model = VA_fusion().to(device)
    VA_model.load_state_dict(VA_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.215, CCC_Arousal: 0.456


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 1.286, mse: 0.45
Validation Loss: 1.298, mse: 0.352
Training Epoch: 2
Train Loss: 1.241, mse: 0.458
Validation Loss: 1.312, mse: 0.349
Training Epoch: 3
Train Loss: 1.221, mse: 0.462
Validation Loss: 1.321, mse: 0.349
Training Epoch: 4
Train Loss: 1.207, mse: 0.464
Validation Loss: 1.328, mse: 0.347
Training Epoch: 5
Train Loss: 1.197, mse: 0.465
Validation Loss: 1.333, mse: 0.346
Training Epoch: 6
Train Loss: 1.189, mse: 0.466
Validation Loss: 1.338, mse: 0.345
Training Epoch: 7
Train Loss: 1.182, mse: 0.467
Validation Loss: 1.343, mse: 0.345
Training Epoch: 8
Train Loss: 1.176, mse: 0.468
Validation Loss: 1.346, mse: 0.345
Training Epoch: 9
Train Loss: 1.171, mse: 0.468
Validation Loss: 1.35, mse: 0.345
Training Epoch: 10
Train Loss: 1.166, mse: 0.469
Validation Loss: 1.354, mse: 0.345
1min 18s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.34, CCC_Arousal: 0.337


##### Effnet

In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss,best_mse, cccv, ccca = train(VA_model, model_type[0], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 0.961, mse: 0.488
Validation Loss: 1.517, mse: 0.328
Training Epoch: 2
Train Loss: 0.859, mse: 0.488
Validation Loss: 1.522, mse: 0.31
Training Epoch: 3
Train Loss: 0.805, mse: 0.487
Validation Loss: 1.546, mse: 0.298
Training Epoch: 4
Train Loss: 0.766, mse: 0.487
Validation Loss: 1.558, mse: 0.288
Training Epoch: 5
Train Loss: 0.735, mse: 0.487
Validation Loss: 1.572, mse: 0.282
Training Epoch: 6
Train Loss: 0.708, mse: 0.487
Validation Loss: 1.584, mse: 0.276
Training Epoch: 7
Train Loss: 0.685, mse: 0.487
Validation Loss: 1.599, mse: 0.275
Training Epoch: 8
Train Loss: 0.664, mse: 0.487
Validation Loss: 1.604, mse: 0.273
Training Epoch: 9
Train Loss: 0.644, mse: 0.487
Validation Loss: 1.619, mse: 0.273
Training Epoch: 10
Train Loss: 0.626, mse: 0.486
Validation Loss: 1.632, mse: 0.274
2min 56s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_fusion_{viau}.pth'))
    VA_model = VA_fusion().to(device)
    VA_model.load_state_dict(VA_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.141, CCC_Arousal: 0.324


In [ ]:
%%timeit -n 1 -r 1
loss_train, loss_val, best_loss, best_mse, cccv, ccca = train(mlp_model, model_type[1], train_loader, val_loader, 10, 32, optimizer, scheduler, auft, viau)

Training Epoch: 1
Train Loss: 1.357, mse: 0.469
Validation Loss: 1.34, mse: 0.352
Training Epoch: 2
Train Loss: 1.315, mse: 0.473
Validation Loss: 1.34, mse: 0.336
Training Epoch: 3
Train Loss: 1.296, mse: 0.474
Validation Loss: 1.338, mse: 0.332
Training Epoch: 4
Train Loss: 1.283, mse: 0.475
Validation Loss: 1.333, mse: 0.329
Training Epoch: 5
Train Loss: 1.274, mse: 0.476
Validation Loss: 1.328, mse: 0.326
Training Epoch: 6
Train Loss: 1.266, mse: 0.477
Validation Loss: 1.323, mse: 0.323
Training Epoch: 7
Train Loss: 1.26, mse: 0.477
Validation Loss: 1.318, mse: 0.32
Training Epoch: 8
Train Loss: 1.254, mse: 0.478
Validation Loss: 1.314, mse: 0.318
Training Epoch: 9
Train Loss: 1.248, mse: 0.478
Validation Loss: 1.311, mse: 0.316
Training Epoch: 10
Train Loss: 1.244, mse: 0.478
Validation Loss: 1.309, mse: 0.314
1min 9s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
try:
    print(f'best metric: CCC_Valence {cccv}, CCC_Arousal: {ccca}')
except:
    mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_mlp_{viau}.pth'))
    mlp_model = MLPModel().to(device)
    mlp_model.load_state_dict(mlp_best_model)
    val_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, val_loader, auft)
    print(f'best metric: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}')

best metric: CCC_Valence 0.376, CCC_Arousal: 0.366


### Testing

##### EffNet + Wav2vec2

In [ ]:
%%timeit -n 1 -r 1
print('VA_model')
print(visual_feat + ' & ' + auft)
VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_fusion_{viau}.pth'))
VA_model = VA_fusion().to(device)
VA_model.load_state_dict(VA_best_model)
test_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, test_loader, auft)
print(f'Test set: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}, mean CCC: {round((ccc1+ccc2)/2,3)}')

VA_model
visualfeat_enet_b2_8_best & audiofeat_wav2vec2
Test set: CCC_Valence 0.294, CCC_Arousal: 0.495, mean CCC: 0.394
58.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_mlp_{viau}.pth'))
mlp_model = MLPModel().to(device)
mlp_model.load_state_dict(mlp_best_model)
test_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, test_loader, auft)
print(f'Test set: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}, mean CCC: {round((ccc1+ccc2)/2,3)}')

MLP_model
visualfeat_enet_b2_8_best & audiofeat_wav2vec2
Test set: CCC_Valence 0.391, CCC_Arousal: 0.527, mean CCC: 0.459
29 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##### EffNet + Vggish

In [ ]:
%%timeit -n 1 -r 1
print('VA_model')
print(visual_feat + ' & ' + auft)
VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_fusion_{viau}.pth'))
VA_model = VA_fusion().to(device)
VA_model.load_state_dict(VA_best_model)
test_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, test_loader, auft)
print(f'Test set: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}, mean CCC: {round((ccc1+ccc2)/2,3)}')

VA_model
visualfeat_enet_b2_8_best & audiofeat_vggish
Test set: CCC_Valence 0.293, CCC_Arousal: 0.458, mean CCC: 0.376
57.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_mlp_{viau}.pth'))
mlp_model = MLPModel().to(device)
mlp_model.load_state_dict(mlp_best_model)
test_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, test_loader, auft)
print(f'Test set: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}, mean CCC: {round((ccc1+ccc2)/2,3)}')

MLP_model
visualfeat_enet_b2_8_best & audiofeat_vggish
Test set: CCC_Valence 0.365, CCC_Arousal: 0.516, mean CCC: 0.44
28.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##### EffNet

In [ ]:
%%timeit -n 1 -r 1
print('VA_model')
print(visual_feat + ' & ' + auft)
VA_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_fusion_{viau}.pth'))
VA_model = VA_fusion().to(device)
VA_model.load_state_dict(VA_best_model)
test_loss, mse, ccc1, ccc2 = evaluate_model(VA_model, test_loader, auft)
print(f'Test set: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}, mean CCC: {round((ccc1+ccc2)/2,3)}')

VA_model
visualfeat_enet_b2_8_best & nope
Test set: CCC_Valence 0.27, CCC_Arousal: 0.377, mean CCC: 0.324
55.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
print('MLP_model')
print(visual_feat + ' & ' + auft)
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_mlp_{viau}.pth'))
mlp_model = MLPModel().to(device)
mlp_model.load_state_dict(mlp_best_model)
test_loss, mse, ccc1, ccc2 = evaluate_model(mlp_model, test_loader, auft)
print(f'Test set: CCC_Valence {ccc1}, CCC_Arousal: {ccc2}, mean CCC: {round((ccc1+ccc2)/2,3)}')

MLP_model
visualfeat_enet_b2_8_best & nope
Test set: CCC_Valence 0.366, CCC_Arousal: 0.464, mean CCC: 0.415
26.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Ensembling

#### EffNet_B2 + Wav2vec2

In [ ]:
# Wav2vec2
vis = vis_typ[0]
auft = audio_feat[0]
viau = vis_aud[0]

In [ ]:
mlp_best_model = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_mlp_{viau}.pth'))
mlp_model_wav = MLPModel().to(device)
mlp_model_wav.load_state_dict(mlp_best_model)
evaluate_model(mlp_model_wav, test_loader, auft)

(1.488, 0.276, 0.391, 0.527)

In [ ]:
iterator = iter(test_loader)
all_preds, all_targets = [], []
img1 = []
for i in range(len(test_loader)):
    VA = next(iterator)
    frame = VA['frame']
    for imgname in frame:
        img1.append(imgname)
    vis_feat, aud_feat, y = VA[visual_feat], VA[auft], VA['label']
    vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
    Vpred, Apred, v_pred, a_pred = mlp_model_wav(vis_feat, aud_feat)
    preds = torch.cat((v_pred, a_pred), dim=1)
    all_preds.extend(preds.cpu().tolist())
    all_targets.extend(y.cpu().tolist())

In [ ]:
compute_VA_CCC(all_preds, all_targets)

(0.3908915320970614, 0.5266098927052195)

#### EffNet_B2 + Vggish

In [ ]:
mlp_best_mod = torch.load(os.path.join(root,f'models/ABAW6/{vis}/best_VA_model/best_VA_mlp_{viau}.pth'))
mlp_model_vgg = MLPModel().to(device)
mlp_model_vgg.load_state_dict(mlp_best_mod)
evaluate_model(mlp_model_vgg, test_loader, auft)

(1.483, 0.287, 0.354, 0.533)

In [ ]:
iterator = iter(test_loader)
all_preds_vgg, all_targets_vgg = [], []
img = []
for i in range(len(test_loader)):
    VA = next(iterator)
    frame = VA['frame']
    for imgname in frame:
        img.append(imgname)
    vis_feat, aud_feat, y = VA[visual_feat], VA[auft], VA['label']
    vis_feat, aud_feat, y = vis_feat.to(device), aud_feat.to(device), y.to(device)
    Vpred, Apred, v_pred, a_pred = mlp_model_vgg(vis_feat, aud_feat)
    preds = torch.cat((v_pred, a_pred), dim=1)
    all_preds_vgg.extend(preds.cpu().tolist())
    all_targets_vgg.extend(y.cpu().tolist())

In [ ]:
compute_VA_CCC(all_preds_vgg, all_targets_vgg)

(0.35368417669976915, 0.53293580621636)

#### Ensemble

In [ ]:
zipped_data = zip(img1, all_preds, all_targets)
zipped_data_vgg = zip(img, all_preds_vgg, all_targets_vgg)

sorted_data = sorted(zipped_data)
sorted_data_vgg = sorted(zipped_data_vgg)

re_img_vgg, re_all_preds_vgg, re_all_targets_vgg = zip(*sorted_data_vgg)
re_img, re_all_preds, re_all_targets = zip(*sorted_data)

In [ ]:
compute_VA_CCC(re_all_preds_vgg, re_all_targets)

(0.3536841766997691, 0.53293580621636)

In [ ]:
for w in np.linspace(0, 1, 11):
    print(f'{w = }')
    w = np.array([w])
    y_ensemble = w * re_all_preds + (1 - w) * re_all_preds_vgg
    cccv, ccca = compute_VA_CCC(y_ensemble, re_all_targets)
    print(f'CCCV: {cccv:.3f}, CCCA: {ccca:.3f}, Mean CCCC: {(cccv+ccca)/2:.3f}')

w = 0.0
CCCV: 0.354, CCCA: 0.533, Mean CCCC: 0.443
w = 0.1
CCCV: 0.364, CCCA: 0.538, Mean CCCC: 0.451
w = 0.2
CCCV: 0.374, CCCA: 0.541, Mean CCCC: 0.458
w = 0.30000000000000004
CCCV: 0.382, CCCA: 0.544, Mean CCCC: 0.463
w = 0.4
CCCV: 0.389, CCCA: 0.545, Mean CCCC: 0.467
w = 0.5
CCCV: 0.393, CCCA: 0.545, Mean CCCC: 0.469
w = 0.6000000000000001
CCCV: 0.396, CCCA: 0.544, Mean CCCC: 0.470
w = 0.7000000000000001
CCCV: 0.398, CCCA: 0.541, Mean CCCC: 0.470
w = 0.8
CCCV: 0.397, CCCA: 0.538, Mean CCCC: 0.467
w = 0.9
CCCV: 0.395, CCCA: 0.533, Mean CCCC: 0.464
w = 1.0
CCCV: 0.391, CCCA: 0.527, Mean CCCC: 0.459


In [ ]:
test_vid = zip(img1, y_ensemble, re_all_targets)

In [ ]:
weight=np.array([0.6])
y_ensemble = weight * re_all_preds + (1 - weight) * re_all_preds_vgg
cccv, ccca = compute_VA_CCC(y_ensemble, re_all_targets)
print(f'CCCV: {cccv:.3f}, CCCA: {ccca:.3f}, Mean CCCC: {(cccv+ccca)/2:.3f}')

CCCV: 0.396, CCCA: 0.544, Mean CCCC: 0.470


### Smoothing

In [ ]:
index, pred, lab = [], [], []
test_vid = {}
for i, val in enumerate(re_img):
    ind = int(val.split('/')[1][:-4])
    vname = val.split('/')[0]
    if i == 0:
        prename = vname
        index.append(ind)
        pred.append(y_ensemble[i])
        lab.append(re_all_targets[i])
    else:
        if vname == prename:
            index.append(ind)
            pred.append(y_ensemble[i])
            lab.append(re_all_targets[i])
        else:
            combined = list(zip(index, pred, lab))
            combined_sorted = sorted(combined, key=lambda x: x[0])
            index_list_sorted, pred_list_sorted, lab_list_sorted = zip(*combined_sorted)
            test_vid[prename] = (list(index_list_sorted), list(pred_list_sorted), list(lab_list_sorted))
            prename = vname
            index, pred, lab = [], [], []
            index.append(ind)
            pred.append(y_ensemble[i])
            lab.append(re_all_targets[i])

In [ ]:
hyperparams=[(isMean,delta) for delta in [0, 5, 15, 30, 60, 100, 200] for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
timing_results = {(isMean, delta): 0 for isMean, delta in hyperparams}
for videoname,(img, predict, label) in test_vid.items():
    for i,ind in enumerate(img):
        total_true.append(label[i])
    preds_proba = smooth_prediction(img, predict)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        preds=[]
        start = time.time()
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            if isMean:
                best_ind, proba = slide_window(preds_proba, i, delta, 'mean')
            else:
                best_ind, proba = slide_window(preds_proba, i, delta, 'median')
            preds.append(proba)
        for i, ind in enumerate(img):
            if label[i][0]>=-1 and label[i][1]>=-1:
                total_preds[hInd].append(preds[ind-1])
        end = time.time()
        timing_results[(isMean, delta)] += end - start
total_true=np.array(total_true)

In [ ]:
hyperparams=[(isMean,delta) for delta in [15, 20, 25, 30, 35, 40] for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
timing_results = {(isMean, delta): 0 for isMean, delta in hyperparams}
for videoname,(img, predict, label) in test_vid.items():
    for i,ind in enumerate(img):
        total_true.append(label[i])
    preds_proba = smooth_prediction(img, predict)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        preds=[]
        start = time.time()
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            if isMean:
                best_ind, proba = slide_window(preds_proba, i, delta, 'mean')
            else:
                best_ind, proba = slide_window(preds_proba, i, delta, 'median')
            preds.append(proba)
        for i, ind in enumerate(img):
            if label[i][0]>=-1 and label[i][1]>=-1:
                total_preds[hInd].append(preds[ind-1])
        end = time.time()
        timing_results[(isMean, delta)] += end - start
total_true=np.array(total_true)

In [ ]:
for hInd, (isMean, delta) in enumerate(hyperparams):
    preds = np.array(total_preds[hInd])
    ccc1, ccc2 = compute_VA_CCC(preds, total_true)
    mean_or_median = 'mean' if isMean else 'median'
    time_taken = round(timing_results[(isMean, delta)],3)
    print(f'{mean_or_median}; delta: {delta}; CCCV: {ccc1:.3f}; CCCA: {ccc2:.3f}; Mean CCC: {(ccc1+ccc2)/2:.3f} Time: {time_taken}s')

mean; delta: 0; CCCV: 0.399; CCCA: 0.546; Mean CCC: 0.473 Time: 5.635s
mean; delta: 5; CCCV: 0.411; CCCA: 0.569; Mean CCC: 0.490 Time: 6.035s
median; delta: 5; CCCV: 0.411; CCCA: 0.567; Mean CCC: 0.489 Time: 13.1s
mean; delta: 15; CCCV: 0.418; CCCA: 0.585; Mean CCC: 0.502 Time: 5.745s
median; delta: 15; CCCV: 0.419; CCCA: 0.584; Mean CCC: 0.502 Time: 13.37s
mean; delta: 30; CCCV: 0.421; CCCA: 0.594; Mean CCC: 0.507 Time: 5.922s
median; delta: 30; CCCV: 0.424; CCCA: 0.596; Mean CCC: 0.510 Time: 13.567s
mean; delta: 60; CCCV: 0.414; CCCA: 0.585; Mean CCC: 0.500 Time: 6.428s
median; delta: 60; CCCV: 0.420; CCCA: 0.594; Mean CCC: 0.507 Time: 14.063s
mean; delta: 100; CCCV: 0.402; CCCA: 0.562; Mean CCC: 0.482 Time: 6.961s
median; delta: 100; CCCV: 0.409; CCCA: 0.575; Mean CCC: 0.492 Time: 14.75s
mean; delta: 200; CCCV: 0.377; CCCA: 0.516; Mean CCC: 0.447 Time: 7.423s
median; delta: 200; CCCV: 0.379; CCCA: 0.531; Mean CCC: 0.455 Time: 16.118s
